In [ ]:
%pip install imblearn streamlit pyngrok joblib scikit-learn pandas numpy

In [ ]:
import pandas as pd
import numpy as np
import joblib
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
from imblearn.over_sampling import SMOTE


In [ ]:
from google.colab import files
uploaded = files.upload()

# Load uploaded dataset
for filename in uploaded.keys():
    df = pd.read_csv(filename)

df.head()


Saving train_data.csv to train_data.csv


,age,sex,on_thyroxine,query_on_thyroxine,on_antithyroid_meds,sick,pregnant,thyroid_surgery,I131_treatment,query_hypothyroid,...,goitre,tumor,hypopituitary,psych,TSH,T3,TT4,T4U,FTI,target
0,70.0,1.0,1,0,0,0,0,0,0,0,...,0,0,0,0,3.0,1.8,95.0,1.00,95.0,0
1,78.0,1.0,0,0,0,0,0,0,0,0,...,0,0,0,0,2.9,1.7,104.0,0.76,137.0,0
2,50.0,1.0,0,0,0,0,0,1,0,0,...,0,0,0,0,3.0,0.0,106.0,1.07,99.0,0
3,44.0,1.0,0,0,0,0,0,0,0,0,...,0,0,0,0,1.2,1.7,92.0,0.98,94.0,0
4,64.0,1.0,1,0,0,0,0,0,0,0,...,0,0,0,0,2.2,0.0,111.0,0.97,114.0,0


In [ ]:
print("Shape of dataset:", df.shape)
print("\nMissing values:\n", df.isnull().sum())
print("\nClass distribution:\n", df['target'].value_counts())


Shape of dataset: (5356, 22)

Missing values:
 age                    0
sex                    0
on_thyroxine           0
query_on_thyroxine     0
on_antithyroid_meds    0
sick                   0
pregnant               0
thyroid_surgery        0
I131_treatment         0
query_hypothyroid      0
query_hyperthyroid     0
lithium                0
goitre                 0
tumor                  0
hypopituitary          0
psych                  0
TSH                    0
T3                     0
TT4                    0
T4U                    0
FTI                    0
target                 0
dtype: int64

Class distribution:
 target
0    4788
1     437
2     131
Name: count, dtype: int64


In [ ]:
# Fill missing values (example strategy)
df = df.fillna(df.median(numeric_only=True))
df = df.fillna(df.mode().iloc[0])


In [ ]:
le = LabelEncoder()
for col in df.select_dtypes(include=['object']).columns:
    df[col] = le.fit_transform(df[col])


In [ ]:
X = df.drop('target', axis=1)   # Replace 'target' with actual target column name
y = df['target']


In [ ]:
smote = SMOTE(random_state=42)
X_res, y_res = smote.fit_resample(X, y)

print("Before SMOTE:", y.value_counts())
print("After SMOTE:", y_res.value_counts())


Before SMOTE: target
0    4788
1     437
2     131
Name: count, dtype: int64
After SMOTE: target
0    4788
1    4788
2    4788
Name: count, dtype: int64


In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X_res, y_res, test_size=0.2, random_state=42)

scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)


In [ ]:
model = RandomForestClassifier(random_state=42)
model.fit(X_train, y_train)


RandomForestClassifier(random_state=42)

In [ ]:
y_pred = model.predict(X_test)
print("Accuracy:", accuracy_score(y_test, y_pred))
print("\nConfusion Matrix:\n", confusion_matrix(y_test, y_pred))
print("\nClassification Report:\n", classification_report(y_test, y_pred))


Accuracy: 0.998259658893143

Confusion Matrix:
 [[963   1   4]
 [  0 949   0]
 [  0   0 956]]

Classification Report:
               precision    recall  f1-score   support

           0       1.00      0.99      1.00       968
           1       1.00      1.00      1.00       949
           2       1.00      1.00      1.00       956

    accuracy                           1.00      2873
   macro avg       1.00      1.00      1.00      2873
weighted avg       1.00      1.00      1.00      2873



In [ ]:
joblib.dump(model, 'thyroid_model.pkl')
joblib.dump(scaler, 'scaler.pkl')


['scaler.pkl']

In [ ]:
import sys, logging, warnings

# --- Completely silence Streamlit's 'missing ScriptRunContext' console spam ---
warnings.filterwarnings("ignore", message=".*ScriptRunContext.*")

# Suppress Streamlit internal debug logging
for name in ["streamlit", "pyngrok.process"]:
    logging.getLogger(name).setLevel(logging.ERROR)

# Also silence stderr (where Streamlit writes these warnings)
class DevNull:
    def write(self, msg): pass
    def flush(self): pass

sys.stderr = DevNull()


In [ ]:
%%writefile thyroid_app.py
import streamlit as st
import pandas as pd
import numpy as np
import joblib
import os

# -------------------------------------------------------
# PAGE CONFIGURATION
# -------------------------------------------------------
st.set_page_config(page_title="Thyroid Disease Detector", layout="wide")


# -------------------------------------------------------
# CUSTOM STYLING
# -------------------------------------------------------
st.markdown("""
    <style>
    .stApp {
        background: linear-gradient(to right, #BADFCD, #F1EDEA);
    }
    .block-container {
        background: transparent !important;
        padding: 2rem 3rem;
    }
    div[data-testid="stTabs"] {
        background-color: transparent !important;
        padding: 10px 0;
        border-bottom: 2px solid #e2e8f0;
    }
    div[data-testid="stTab"] {
        font-family: 'Segoe UI', sans-serif;
        font-size: 18px;
        font-weight: 600;
        color: #1e293b;
        background-color: #e2e8f0;
        border-radius: 12px 12px 0 0;
        margin-right: 12px;
        padding: 10px 24px;
        transition: all 0.3s ease;
        box-shadow: 0 2px 4px rgba(0,0,0,0.05);
    }
    div[data-testid="stTab"]:hover {
        background-color: #cbd5e1;
        color: #0f172a;
    }
    div[data-testid="stTab"][aria-selected="true"] {
        background-color: #3b82f6;
        color: white;
        box-shadow: 0 4px 6px rgba(0,0,0,0.1);
    }
    .stTextInput > div > div > input,
    .stNumberInput > div > div > input,
    .stSelectbox > div > div,
    .stRadio > div,
    .stTextArea > div > textarea {
        background-color: #f8fafc !important;
        border-radius: 10px !important;
    }
    h1, h2, h3 {
        color: #1e3a8a;
        font-family: 'Segoe UI', sans-serif;
    }
    p, li {
        color: #334155;
        font-size: 16px;
    }
    </style>
""", unsafe_allow_html=True)

# -------------------------------------------------------
# LOAD MODEL AND SCALER
# -------------------------------------------------------
@st.cache_resource
def load_artifacts():
    if not os.path.exists("thyroid_model.pkl"):
        st.error("Model file ('thyroid_model.pkl') is missing.")
        st.stop()
    if not os.path.exists("scaler.pkl"):
        st.warning("Scaler file ('scaler.pkl') not found. Proceeding without scaling.")
        scaler = None
    else:
        scaler = joblib.load("scaler.pkl")

    model = joblib.load("thyroid_model.pkl")
    label_map = {0: "No Thyroid Disorder", 1: "Hypothyroid", 2: "Hyperthyroid"}
    return model, scaler, label_map

model, scaler, label_map = load_artifacts()

# -------------------------------------------------------
# TOPBAR NAVIGATION
# -------------------------------------------------------
tabs = st.tabs(["Home", "Single-Patient Thyroid Analysis", "Multi-Patient Thyroid Analysis", "About REMI", "Learn More", "Wellness Hub", "Contact"])

st.markdown("""
    <div style='text-align:right; font-size:14px; margin-top:-10px;'>
        Prefer dark mode? Go to <b>Settings → Theme</b> and select <i>Dark</i>.
    </div>
""", unsafe_allow_html=True)


# -------------------------------------------------------
# HOME PAGE
# -------------------------------------------------------
with tabs[0]:
    # --- Hero Section ---
    st.markdown("""
        <div style='text-align: center; padding: 30px 0 10px 0;'>
            <h1 style='font-size: 42px; color: #228B22; margin-bottom: 10px;'>REMI – The Thyroid Disease Detector</h1>
            <p style='font-size:18px;'>
        <span style='color:#2E8B57;'>
                Empowering early thyroid detection through AI-driven insights.
                </span>
            </p>
        </div>
    """, unsafe_allow_html=True)

    st.markdown("<hr style='margin:25px 0;'>", unsafe_allow_html=True)

    # --- About REMI ---
    st.markdown("""
        <h2 style='color:#1e3a8a;'>About REMI</h2>
        <p style='font-size:16px; color:#334155;'>
        REMI (Rapid Endocrine Monitoring Interface) is an intelligent
        web-based application designed to assist in the early identification and
        assessment of thyroid disorders using machine learning models.
By analyzing key biochemical parameters from routine blood tests,
REMI classifies patient data as <b>normal</b>, <b>hypothyroid</b>, or <b>hyperthyroid</b>,
helping clinicians and users gain quick, data-based insights.

The tool provides an accessible platform for preliminary evaluation and
awareness, supporting faster decisions and encouraging timely medical consultation when abnormalities are detected. </p>
    """, unsafe_allow_html=True)

    # --- About Thyroid Section ---
    st.markdown("""
        <h2 style='color:#1e3a8a; margin-top:30px;'>Thyroid at a glance</h2>
        <p style='font-size:16px; color:#334155;'>

          The thyroid gland is a small, butterfly-shaped organ located at the base of the neck. It produces
            hormones such as T3 and T4, which regulate metabolism, growth, and energy levels throughout the body.
            An imbalance in thyroid hormone levels can lead to conditions like <b>hypothyroidism</b> (underactive thyroid)
            or <b>hyperthyroidism</b> (overactive thyroid), each with distinct clinical symptoms.
            Early detection through diagnostic indicators like TSH, TT4, and FTI can significantly improve health outcomes.
        </p>
    """, unsafe_allow_html=True)

    # --- Early Detection Section ---
    st.markdown("""
        <h2 style='color:#1e3a8a; margin-top:20px;'>Why Early Detection Matters</h2>
        <p style='font-size:16px; color:#334155;'>
            Timely screening of thyroid levels helps prevent complications such as fatigue, weight changes, and mood imbalance.
            AI-powered tools like REMI make it easier to spot warning signs early, empowering individuals to seek medical guidance sooner.
        </p>
    """, unsafe_allow_html=True)


    # --- Features Section ---
    st.markdown("""
        <h2 style='color:#1e3a8a; margin-top:30px;'>Core Features</h2>
        <div style='display:flex; flex-wrap:wrap; justify-content:space-around; margin-top:15px;'>
            <div style='background-color:#F8FAFC; padding:20px; border-radius:12px; width:45%; margin:10px; box-shadow:0 2px 6px rgba(0,0,0,0.1);'>
                <h3 style='color:#1e40af;'>Single Prediction</h3>
                <p>Input one patient’s lab test data to instantly predict thyroid condition and confidence level.</p>
            </div>
            <div style='background-color:#F8FAFC; padding:20px; border-radius:12px; width:45%; margin:10px; box-shadow:0 2px 6px rgba(0,0,0,0.1);'>
                <h3 style='color:#1e40af;'>Batch Prediction</h3>
                <p>Upload a CSV file containing multiple patient records to generate predictions in bulk.</p>
            </div>
            <div style='background-color:#F8FAFC; padding:20px; border-radius:12px; width:45%; margin:10px; box-shadow:0 2px 6px rgba(0,0,0,0.1);'>
                <h3 style='color:#1e40af;'>Confidence Insights</h3>
                <p>Each prediction includes a confidence score based on model probability values.</p>
            </div>
            <div style='background-color:#F8FAFC; padding:20px; border-radius:12px; width:45%; margin:10px; box-shadow:0 2px 6px rgba(0,0,0,0.1);'>
                <h3 style='color:#1e40af;'>User-Friendly Design</h3>
                <p>A clean, intuitive interface that makes clinical AI integration simple and efficient.</p>
            </div>
        </div>
    """, unsafe_allow_html=True)

    # --- Get Started Section ---
    st.markdown("""
        <div style='text-align:center; margin-top:40px;'>
            <p style='color:#475569; font-size:14px; margin-top:8px;'>
                Use the tabs above to navigate to Single or Batch Prediction.
            </p>
        </div>
    """, unsafe_allow_html=True)

    st.success("Model and Scaler loaded successfully!")



# -------------------------------------------------------
# SINGLE PREDICTION PAGE
# -------------------------------------------------------
    with tabs[1]:
        st.markdown("<h1 style='text-align:center; color:#228B22; margin-bottom:10px;'>Single-Patient Thyroid Analysis </h1>", unsafe_allow_html=True)
        st.markdown("<hr style='border:1px solid #ddd'>", unsafe_allow_html=True)
        st.markdown("<p>Enter the details below as per the patient’s lab results.</p>", unsafe_allow_html=True)

        # --- Numeric inputs ---
        st.markdown("<h3 style='text-align:center;'>Lab Results & Basic Info</h3>", unsafe_allow_html=True)

        age = st.number_input("Age", 0, 120, 35)
        sex = st.selectbox("Sex", ["Female", "Male"])
        sex = 1 if sex == "Male" else 0

        TSH = st.number_input("TSH", 0.0, 500.0, 2.5)
        T3 = st.number_input("T3", 0.0, 10.0, 3.0)
        TT4 = st.number_input("TT4", 0.0, 500.0, 110.0)
        T4U = st.number_input("T4U", 0.0, 10.0, 1.0)
        FTI = st.number_input("FTI", 0.0, 500.0, 120.0)

            # --- Medical History ---
        st.markdown("<h3 style='text-align:center;'>Medical History</h3>", unsafe_allow_html=True)

        def yn(label):
            return st.radio(label, ["No", "Yes"], horizontal=True, key=label)

        # Use two evenly spaced columns across the full width
        col1, col2 = st.columns([1, 1], gap="large")

        with col1:
            on_thyroxine = 1 if yn("On Thyroxine") == "Yes" else 0
            query_on_thyroxine = 1 if yn("Query Thyroxine") == "Yes" else 0
            on_antithyroid_meds = 1 if yn("On Anti-thyroid Meds") == "Yes" else 0
            sick = 1 if yn("Recently Ill") == "Yes" else 0
            pregnant = 1 if yn("Pregnant") == "Yes" else 0
            thyroid_surgery = 1 if yn("Thyroid Surgery") == "Yes" else 0
            I131_treatment = 1 if yn("I131 Treatment") == "Yes" else 0

        with col2:

            query_hypothyroid = 1 if yn("Query Hypothyroid") == "Yes" else 0
            query_hyperthyroid = 1 if yn("Query Hyperthyroid") == "Yes" else 0
            lithium = 1 if yn("Lithium") == "Yes" else 0
            goitre = 1 if yn("Goitre") == "Yes" else 0
            tumor = 1 if yn("Tumor") == "Yes" else 0
            hypopituitary = 1 if yn("Hypopituitary") == "Yes" else 0
            psych = 1 if yn("Psychiatric History") == "Yes" else 0



        # --- Prepare input data ---
        input_data = {
            "age": age, "sex": sex, "on_thyroxine": on_thyroxine, "query_on_thyroxine": query_on_thyroxine,
            "on_antithyroid_meds": on_antithyroid_meds, "sick": sick, "pregnant": pregnant,
            "thyroid_surgery": thyroid_surgery, "I131_treatment": I131_treatment,
            "query_hypothyroid": query_hypothyroid, "query_hyperthyroid": query_hyperthyroid,
            "lithium": lithium, "goitre": goitre, "tumor": tumor, "hypopituitary": hypopituitary,
            "psych": psych, "TSH": TSH, "T3": T3, "TT4": TT4, "T4U": T4U, "FTI": FTI
        }

        input_df = pd.DataFrame([input_data])
        if hasattr(model, "feature_names_in_"):
            for col in model.feature_names_in_:
                if col not in input_df.columns:
                    input_df[col] = 0
            input_df = input_df[model.feature_names_in_]

        # --- Prediction button ---
        if st.button("Predict Result", type="primary"):
            try:
                if scaler is not None:
                    input_df[input_df.columns] = scaler.transform(input_df)
                pred = model.predict(input_df)[0]
                prob = model.predict_proba(input_df)[0]
                confidence = round(max(prob) * 100, 2)

                st.success(f"### Prediction: {label_map[pred]}")
                st.info(f"Confidence Level: **{confidence}%**")

                prob_df = pd.DataFrame({
                    "Condition": [label_map[i] for i in range(len(prob))],
                    "Probability (%)": [round(p * 100, 2) for p in prob]
                })
                st.table(prob_df)

            except Exception as e:
                st.error(f"Error during prediction: {e}")


# -------------------------------------------------------
# BATCH PREDICTION PAGE
# -------------------------------------------------------
with tabs[2]:
    st.markdown("<h1 style='text-align:center; color:#228B22; margin-bottom:10px;'>Multi-Patient Thyroid Analysis</h1>", unsafe_allow_html=True)
    st.markdown("<hr style='border:1px solid #ddd'>", unsafe_allow_html=True)

    uploaded_file = st.file_uploader("📤 Upload CSV File", type=["csv"])

    if uploaded_file is not None:
        try:
            df = pd.read_csv(uploaded_file)

            # ✅ FIX: Drop target column if present
            if "target" in df.columns:
                df = df.drop(columns=["target"])

            st.markdown("<h4 style='margin-top:20px;'> Data Preview</h4>", unsafe_allow_html=True)
            st.dataframe(df.head())

            if hasattr(model, "feature_names_in_"):
                missing_cols = [col for col in model.feature_names_in_ if col not in df.columns]
                if missing_cols:
                    st.warning(f"The following columns are missing and will be filled with zeros: {missing_cols}")
                    for col in missing_cols:
                        df[col] = 0
                df = df[model.feature_names_in_]

            if scaler is not None:
                df[df.columns] = scaler.transform(df)

            if st.button(" Run Batch Prediction"):
                preds = model.predict(df)
                probs = model.predict_proba(df)
                labels = [label_map[p] for p in preds]
                confidence = [f"{round(max(prob)*100,2)}%" for prob in probs]

                results_df = df.copy()
                results_df["Prediction"] = labels
                results_df["Confidence"] = confidence

                st.success("✅ Batch prediction completed successfully!")
                st.markdown("<h4>🧾 Prediction Results</h4>", unsafe_allow_html=True)
                st.dataframe(results_df)

                csv = results_df.to_csv(index=False).encode('utf-8')
                st.download_button(
                    label="📥 Download Results as CSV",
                    data=csv,
                    file_name="thyroid_batch_predictions.csv",
                    mime="text/csv",
                )
        except Exception as e:
            st.error(f"Error processing file: {e}")
    else:
        st.info("Please upload a CSV file to begin batch prediction.")

# -------------------------------------------------------
# ABOUT PAGE
# -------------------------------------------------------
with tabs[3]:
    st.markdown("<h1 style='text-align:center; color:#228B22; margin-bottom:1px;'>About This Application</h1>", unsafe_allow_html=True)
    st.markdown("<hr style='border:1px solid #ddd'>", unsafe_allow_html=True)

    st.markdown("""
    The Thyroid Disease Detection System is an AI-powered web application designed to facilitate the early detection and preliminary assessment of thyroid disorders. By leveraging advanced data analytics and predictive modeling, it evaluates key biochemical markers from routine blood tests—such as TSH, T3, TT4, T4U, and FTI—to provide clear insights into whether an individual may have hypothyroidism, hyperthyroidism, or normal thyroid function.

This platform is created to support both healthcare professionals and patients by delivering fast, reliable, and interpretable results. It bridges the gap between conventional laboratory diagnostics and modern AI-driven decision support, allowing for quicker recognition of potential thyroid abnormalities and timely medical intervention.

The system highlights the integration of artificial intelligence into clinical practice, offering data-driven assistance to doctors, laboratories, and research institutions. Its predictive capabilities help medical personnel make informed decisions, prioritize patient care, and identify cases that require closer attention. Beyond diagnostics, the platform serves as an educational resource, enhancing awareness of thyroid health and promoting proactive management of potential issues.

By combining accuracy, accessibility, and usability, the Thyroid Disease Detection System demonstrates how technology can enhance traditional healthcare workflows, improve diagnostic efficiency, and maintain patient safety and reliability.
           """, unsafe_allow_html=True)

    st.markdown("<h3>System Overview</h3>", unsafe_allow_html=True )
    st.markdown("""
    The platform analyzes key biochemical parameters, including TSH, T3, TT4, T4U, and FTI, along with patient-specific details such as age, sex, and relevant medical history. Leveraging a trained predictive model, it classifies patient data into clear diagnostic categories, providing an evidence-based assessment of thyroid function.

The application offers two main modes of operation:

1. Single Prediction: Users can manually enter individual patient values to receive an immediate diagnostic prediction. This feature is perfect for quick evaluations or one-off assessments, delivering actionable insights in real time.

2. Batch Prediction: Users can upload a CSV file containing multiple patient records, enabling the system to process all entries simultaneously. Each output includes the predicted thyroid condition and a confidence score, indicating the model’s reliability for that prediction.

By supporting both single and batch analyses, the system ensures accuracy, efficiency, and accessibility, making it a valuable tool for healthcare professionals, laboratories, and research teams alike.

           """, unsafe_allow_html=True)

    st.markdown("<h3>Core Technologies</h3>", unsafe_allow_html=True)
    st.markdown("""
    The Thyroid Disease Detection System is built using a combination of robust technologies to ensure accuracy, scalability, and seamless user experience.

Python & Streamlit: Python provides a versatile foundation for data processing and machine learning, while Streamlit enables the creation of a highly interactive web application, allowing users to input data, view results, and navigate features effortlessly.

Scikit-learn: This widely-used machine learning library powers the predictive models in the system. It supports model training, validation, and evaluation, ensuring predictions are both accurate and interpretable.

Pandas & NumPy: These libraries allow efficient handling, transformation, and analysis of patient datasets. They ensure both single and batch inputs are processed quickly and reliably while maintaining data integrity.
   """, unsafe_allow_html=True)

    st.markdown("<h3>Design Philosophy and Objective</h3>", unsafe_allow_html=True)
    st.markdown("""
The design of this system prioritizes accuracy, interpretability, and ease of use, recognizing that reliable predictions and user clarity are essential in healthcare applications. Users—including clinicians, researchers, and patients—can understand and trust the results, ensuring that insights are not only precise but also actionable.

This platform illustrates how predictive analytics can complement traditional laboratory diagnostics. Rather than replacing clinical workflows, it provides rapid, data-driven insights that support faster and better-informed decision-making. By analyzing biochemical and patient-specific parameters with advanced algorithms, the system can flag potential thyroid issues early, enabling timely intervention and improved patient outcomes.

The tool is intended for research, education, and clinical support environments. It enhances early detection of thyroid disorders, improves diagnostic efficiency, and empowers healthcare professionals to interpret patient data comprehensively. Laboratories benefit from streamlined processing of large datasets, and students or practitioners gain exposure to practical AI applications in medicine.

It is important to note that this system does not replace professional medical evaluation or laboratory testing. Instead, it serves as a supportive technology that augments human expertise, helping clinicians prioritize cases, verify findings, and deliver timely care while promoting awareness of thyroid health.   """, unsafe_allow_html=True)



# -------------------------------------------------------
# LEARN PAGE
# -------------------------------------------------------
with tabs[4]:
    st.markdown("<h1 style='text-align:center; color:#228B22; margin-bottom:10px;'>About REMI - The Thyroid Disease Detector</h1>", unsafe_allow_html=True)
    st.markdown("<hr style='border:1px solid #ddd'>", unsafe_allow_html=True)
    # custom styling for headings, text color, and boxes
    st.markdown("""
        <style>
        h2, h3, h4 {
            color: #015A84 !important;
        }
        body, p, li, td, th {
            color: #1E293B !important;  /* deep gray-blue text */
        }
        .info-box {
            border: 1px solid #000;
            border-radius: 10px;
            background-color: rgba(255, 255, 255, 0.6);
            padding: 15px;
            margin-top: 10px;
            margin-bottom: 20px;
        }
        table {
            width: 100%;
            border-collapse: collapse;
            margin-top: 10px;
            color: #1E293B;
        }
        th, td {
            border: 1px solid #000;
            padding: 8px;
            text-align: left;
        }
        th {
            background-color: rgba(1, 90, 132, 0.1);
            color: #015A84;
        }
        ul {
            color: #1E293B;
        }
        </style>
    """, unsafe_allow_html=True)

    st.markdown("<h2>Learn About Thyroid Disorders</h2>", unsafe_allow_html=True)

    st.markdown("""
    The thyroid gland plays a vital role in regulating metabolism, energy, and hormone balance. Proper thyroid function influences heart rate, digestion, body temperature, and cognitive performance. Early understanding of thyroid disorders helps identify symptoms promptly, enabling better management and preventing potential complications. Awareness of thyroid health empowers individuals to monitor changes, seek timely medical advice, and adopt lifestyle or treatment interventions that support overall well-being and long-term health outcomes.

    ### Common Thyroid Conditions
    - **Hypothyroidism:** The thyroid produces too little hormone.
    Symptoms: fatigue, weight gain, cold intolerance, depression.
    - **Hyperthyroidism:** The thyroid produces too much hormone.
    Symptoms: weight loss, anxiety, tremors, heat intolerance.
    - **Goitre:** Enlargement of the thyroid gland, can occur with or without hormone imbalance.
    """, unsafe_allow_html=True)

    # Common Symptoms (no box)
    st.markdown("""
    <h3>Common Symptoms of Thyroid Imbalance</h3>
    <table>
    <tr><th>Type</th><th>Symptoms</th></tr>
    <tr><td>Hypothyroidism</td><td>Fatigue, dry skin, weight gain, hair loss, feeling cold, slow heart rate</td></tr>
    <tr><td>Hyperthyroidism</td><td>Weight loss, irritability, sweating, rapid heartbeat, trembling hands</td></tr>
    </table>
    """, unsafe_allow_html=True)

    # Key Blood Tests box
    st.markdown("""
    <div class="info-box">
    <h3>Key Blood Tests</h3>
    <table>
        <tr>
            <th>Test</th><th>Measures</th><th>Typical Range</th><th>Meaning</th>
        </tr>
        <tr>
            <td><i>TSH</i></td><td>Thyroid Stimulating Hormone</td><td>0.4 – 4.0 mIU/L</td><td>High = Hypothyroid; Low = Hyperthyroid</td>
        </tr>
        <tr>
            <td><i>T3</i></td><td>Triiodothyronine</td><td>0.8 – 2.0 ng/mL</td><td>Active thyroid hormone</td>
        </tr>
        <tr>
            <td><i>TT4</i></td><td>Total Thyroxine</td><td>60 – 150 µg/dL</td><td>Total circulating T4</td>
        </tr>
        <tr>
            <td><i>T4U</i></td><td>Thyroxine Uptake</td><td>0.7 – 1.8</td><td>Measures T4-binding capacity</td>
        </tr>
        <tr>
            <td><i>FTI</i></td><td>Free Thyroxine Index</td><td>80 – 180</td><td>Estimates free thyroid hormone levels</td>
        </tr>
    </table>
    </div>
    """, unsafe_allow_html=True)

    # Thyroid diet box
    st.markdown("""
    <div class="info-box">
    <h3>Healthy Diet for Thyroid Health (Indian Menu)</h3>
    A balanced diet supports thyroid function and helps manage symptoms effectively.
    Focus on nutrient-rich foods and avoid those that interfere with hormone absorption.

    <h4>What to Include</h4>
    <table>
        <tr><th>Category</th><th>Examples</th></tr>
        <tr><td>Iodine-rich foods</td><td>Iodized salt, seaweed (moderation), curd</td></tr>
        <tr><td>Protein sources</td><td>Moong dal, lentils, paneer, eggs, chicken, fish</td></tr>
        <tr><td>Selenium & Zinc</td><td>Sunflower seeds, pumpkin seeds, cashews, whole grains</td></tr>
        <tr><td>Fibre & Antioxidants</td><td>Fruits (papaya, apple, pomegranate), vegetables (palak, lauki, beans)</td></tr>
        <tr><td>Healthy fats</td><td>Ghee (in moderation), olive oil, nuts, seeds</td></tr>
    </table>

    <h4>Foods to Limit or Avoid</h4>
    <table>
        <tr><th>Type</th><th>Examples</th></tr>
        <tr><td>Goitrogenic foods (excess)</td><td>Raw cabbage, cauliflower, broccoli (cook before eating)</td></tr>
        <tr><td>Soy products</td><td>Soy chunks, tofu, soy milk</td></tr>
        <tr><td>Refined sugar & carbs</td><td>Pastries, sweets, white bread</td></tr>
        <tr><td>Caffeine & alcohol</td><td>Limit intake to avoid absorption issues</td></tr>
    </table>

    <h4>Sample One-Day Thyroid-Friendly Meal Plan</h4>
    <table>
        <tr><th>Time</th><th>Meal</th></tr>
        <tr><td>Morning (empty stomach)</td><td>Thyroid medication + lukewarm water</td></tr>
        <tr><td>Breakfast</td><td>Vegetable oats upma + 1 boiled egg / glass of milk</td></tr>
        <tr><td>Mid-morning</td><td>Fruit bowl (papaya or apple)</td></tr>
        <tr><td>Lunch</td><td>Brown rice / phulka + dal + palak sabzi + curd</td></tr>
        <tr><td>Evening</td><td>Roasted chana / handful of nuts</td></tr>
        <tr><td>Dinner</td><td>Moong dal khichdi / grilled paneer + sautéed veggies</td></tr>
        <tr><td>Before bed</td><td>Warm turmeric milk (optional)</td></tr>
    </table>
    </div>
    """, unsafe_allow_html=True)

    # Everyday healthy diet box
    st.markdown("""
    <div class="info-box">
    <h3>Everyday Healthy Diet (Indian Style)</h3>
    <p>Maintaining a balanced diet helps support energy, immunity, and long-term wellness.</p>

    <h4>Basic Guidelines</h4>
    <ul>
        <li>Eat fresh, home-cooked meals with balanced portions.</li>
        <li>Include all food groups: whole grains, proteins, healthy fats, fruits, and vegetables.</li>
        <li>Avoid overeating, processed snacks, and sugary drinks.</li>
        <li>Stay hydrated and active throughout the day.</li>
    </ul>

    <h4>Ideal Daily Indian Meal Plan</h4>
    <table>
        <tr><th>Time</th><th>Meal</th></tr>
        <tr><td>Early Morning</td><td>Warm water with lemon / soaked almonds (4–5)</td></tr>
        <tr><td>Breakfast</td><td>Poha / vegetable paratha with curd / idli with sambar</td></tr>
        <tr><td>Mid-Morning</td><td>Seasonal fruit (banana, apple, orange)</td></tr>
        <tr><td>Lunch</td><td>Brown rice / 2 rotis + dal + sabzi + salad + curd</td></tr>
        <tr><td>Evening Snack</td><td>Sprouts chaat / roasted makhana / green tea</td></tr>
        <tr><td>Dinner</td><td>Light meal – khichdi / vegetable soup + grilled paneer or tofu</td></tr>
        <tr><td>Before Bed</td><td>Warm milk with turmeric (optional)</td></tr>
    </table>

    <h4>Healthy Habits</h4>
    <ul>
        <li>Drink 8–10 glasses of water daily.</li>
        <li>Limit fried food, sugar, and packaged juices.</li>
        <li>Practice mindful eating — eat slowly and avoid screens.</li>
        <li>Get 7–8 hours of quality sleep.</li>
    </ul>
    </div>
    """, unsafe_allow_html=True)

    # Quick Do’s & Don’ts (no box)
    st.markdown("""
    <h3>Quick Do’s & Don’ts</h3>
    <table>
    <tr><th>Do</th><th>Don’t</th></tr>
    <tr><td>Take thyroid medication on an empty stomach</td><td>Skip doses or take with coffee/milk</td></tr>
    <tr><td>Eat iodine and selenium-rich foods</td><td>Overconsume raw cruciferous veggies</td></tr>
    <tr><td>Exercise moderately</td><td>Ignore fatigue or sudden mood swings</td></tr>
    </table>
    """, unsafe_allow_html=True)

    # Disclaimer
    st.markdown("""
    ### Medical Disclaimer
    This app is for educational and research purposes only.
    It should not be used for self-diagnosis or treatment decisions.
    Always consult a licensed healthcare provider for clinical interpretation.
    """, unsafe_allow_html=True)



# -------------------------------------------------------
# WELLNESS HUB PAGE
# -------------------------------------------------------
with tabs[5]:
    st.markdown("<h1 style='text-align:center; color:#228B22; margin-bottom:10px;'>Wellness Hub – Find Trusted Thyroid Specialists in India</h1>", unsafe_allow_html=True)
    st.markdown("<hr style='border:1px solid #ddd'>", unsafe_allow_html=True)
    st.markdown("""
        <p style='font-size:16px; color:#475569;'>
            Discover top hospitals and endocrinology specialists across India specializing in thyroid care and hormonal health.
            Use this directory to find reliable medical consultation and advanced diagnostic facilities.
        </p>
    """, unsafe_allow_html=True)

    hospitals = [
        {
            "city": "New Delhi",
            "hospital": "All India Institute of Medical Sciences (AIIMS)",
            "doctor": "Dr. Ramesh Goyal – Endocrinologist",
            "address": "Ansari Nagar, New Delhi – 110029",
            "contact": "+91 11 2658 8500"
        },
        {
            "city": "Mumbai",
            "hospital": "Fortis Hospital, Mulund",
            "doctor": "Dr. Shashank Joshi – Senior Consultant, Endocrinology",
            "address": "Mulund Goregaon Link Rd, Mumbai – 400078",
            "contact": "+91 22 6799 3333"
        },
        {
            "city": "Chennai",
            "hospital": "Apollo Hospital",
            "doctor": "Dr. V. Mohan – Chief Diabetologist & Endocrinologist",
            "address": "Greams Lane, Off Greams Road, Chennai – 600006",
            "contact": "+91 44 2829 3333"
        },
        {
            "city": "Bengaluru",
            "hospital": "Manipal Hospital",
            "doctor": "Dr. Suresh Raghavan – Consultant Endocrinologist",
            "address": "Old Airport Road, Bengaluru – 560017",
            "contact": "+91 80 2222 1111"
        },
        {
            "city": "Kolkata",
            "hospital": "AMRI Hospitals",
            "doctor": "Dr. Anirban Majumder – Endocrinologist",
            "address": "JC-16 & 17, Salt Lake City, Kolkata – 700098",
            "contact": "+91 33 6680 0000"
        },
        {
            "city": "Hyderabad",
            "hospital": "Yashoda Hospitals",
            "doctor": "Dr. Nitin Kapoor – Consultant Endocrinologist",
            "address": "Raj Bhavan Road, Somajiguda, Hyderabad – 500082",
            "contact": "+91 40 4567 4567"
        },
        {
            "city": "Pune",
            "hospital": "Ruby Hall Clinic",
            "doctor": "Dr. Rajesh Rajput – Thyroid & Hormone Specialist",
            "address": "40, Sassoon Road, Pune – 411001",
            "contact": "+91 20 2616 3391"
        },
        {
            "city": "Ahmedabad",
            "hospital": "Sterling Hospital",
            "doctor": "Dr. Harshal Joshi – Consultant Endocrinologist",
            "address": "Gurukul Road, Memnagar, Ahmedabad – 380052",
            "contact": "+91 79 4001 1111"
        }
    ]

    # Style the cards
    st.markdown("""
        <style>
        .hospital-card {
            background-color: #F1EDEA;
            border-radius: 12px;
            padding: 18px 22px;
            margin-bottom: 16px;
            box-shadow: 0 2px 6px rgba(0,0,0,0.08);
            border: 1px solid #e2e8f0;
        }
        .hospital-card h4 {
            margin: 0;
            color: #1e3a8a;
        }
        .hospital-card p {
            margin: 4px 0;
            color: #334155;
        }
        </style>
    """, unsafe_allow_html=True)

    for entry in hospitals:
        st.markdown(f"""
            <div class='hospital-card'>
                <h4>{entry['hospital']} ({entry['city']})</h4>
                <p><b>Specialist:</b> {entry['doctor']}</p>
                <p><b>Address:</b> {entry['address']}</p>
                <p><b>Contact:</b> {entry['contact']}</p>
            </div>
        """, unsafe_allow_html=True)


# -------------------------------------------------------
# CONTACT PAGE
# -------------------------------------------------------
with tabs[6]:
    st.markdown("<h1 style='text-align:center; color:#228B22; margin-bottom:10px;'>Have Questions? Contact Us!</h1>", unsafe_allow_html=True)
    st.markdown("<hr style='border:1px solid #ddd'>", unsafe_allow_html=True)
    st.markdown("""
        <style>
        /* ===== Input fields (Your Name, Your Email) ===== */
        div[data-baseweb="input"] {
            background-color: #F1EDEA !important;
            border-radius: 10px !important;
            border: 1px solid #d1d5db !important;
            box-shadow: 0px 2px 4px rgba(0,0,0,0.05) !important;
        }
        div[data-baseweb="input"] input {
            background-color: #F1EDEA !important;
            color: #1e293b !important;
            border: none !important;
        }

        /* ===== Text area (Your Message) ===== */
        div[data-baseweb="textarea"] {
            background-color: #ffffff !important;   /* white box look */
            border-radius: 10px !important;
            border: 1px solid #d1d5db !important;
            box-shadow: 0px 2px 6px rgba(0,0,0,0.08) !important;
        }
        div[data-baseweb="textarea"] textarea {
            background-color: #ffffff !important;
            color: #1e293b !important;
            border: none !important;
            border-radius: 10px !important;
            padding: 10px !important;
        }

        /* ===== Button styling ===== */
        .stForm button[kind="primary"] {
            background-color: #F1EDEA !important;
            color: #1e293b !important;
            border: 1px solid #cbd5e1 !important;
            border-radius: 10px !important;
            transition: all 0.2s ease-in-out !important;
        }
        .stForm button[kind="primary"]:hover {
            background-color: #e7deda !important;
            color: #111827 !important;
            border-color: #94a3b8 !important;
        }

        /* ===== Header and list styling ===== */
        h2, h3 { color: #1e293b !important; }

        .team-list {
            list-style-type: none;
            padding-left: 0;
        }
        .team-list li {
            background-color: #F1EDEA;
            padding: 12px 18px;
            margin-bottom: 10px;
            border-radius: 10px;
            font-size: 16px;
            color: #1e293b;
            display: flex;
            justify-content: space-between;
            align-items: center;
            border: 1px solid #d1d5db;
            box-shadow: 0 2px 4px rgba(0,0,0,0.05);
        }

        /* ===== LinkedIn button styling ===== */
        .linkedin-btn {
            background-color: #0077B5;
            color: white !important;
            text-decoration: none;
            padding: 6px 12px;
            border-radius: 6px;
            font-size: 14px;
            font-weight: 500;
            transition: all 0.2s ease-in-out;
        }
        .linkedin-btn:hover {
            background-color: #005b86;
        }
        </style>
    """, unsafe_allow_html=True)

    # --- Contact form section ---
    st.markdown("<h2>Contact Us</h2>", unsafe_allow_html=True)
    with st.form("contact_form"):
        name = st.text_input("Your Name")
        email = st.text_input("Your Email")
        message = st.text_area("Your Message")
        submitted = st.form_submit_button("Send Message")
        if submitted:
            st.success("✅ Thank you! Your message has been received.")

    # --- Team section with LinkedIn buttons ---
    st.markdown("""
        <h3>Connect With Us</h3>
        <ul class="team-list">
            <li><span><b>Ishika Das</b> – 2205289</span>
                <a class="linkedin-btn" href="https://www.linkedin.com/in/ishika-das-39aa2324a/" target="_blank">LinkedIn</a>
            </li>
            <li><span><b>Anishka</b> – 2205356</span>
                <a class="linkedin-btn" href="https://www.linkedin.com/in/anishka2011/" target="_blank">LinkedIn</a>
            </li>
            <li><span><b>Sangbarta Sarkar</b> – 22051612</span>
                <a class="linkedin-btn" href="https://www.linkedin.com/in/sangbartasarkar02" target="_blank">LinkedIn</a>
            </li>
            <li><span><b>Aryan Pattnaik</b> – 2228018</span>
                <a class="linkedin-btn" href="https://www.linkedin.com/in/aryan-pattnaik/?originalSubdomain=in" target="_blank">LinkedIn</a>
            </li>
        </ul>
    """, unsafe_allow_html=True)


# --- Footer ---
st.markdown("""
    <hr style='margin-top:50px;'>
    <div style='text-align: center; font-size: 13px; color: #64748b;'>
        © 2025 <b>REMI</b> | Powered by Streamlit and Machine Learning
    </div>
""", unsafe_allow_html=True)

Writing thyroid_app.py


In [ ]:
%%writefile thyroid_app.py
import streamlit as st
import pandas as pd
import numpy as np
import joblib
import os

# -------------------------------------------------------
# PAGE CONFIGURATION
# -------------------------------------------------------
st.set_page_config(page_title="Thyroid Disease Detector", layout="wide")


# -------------------------------------------------------
# CUSTOM STYLING
# -------------------------------------------------------
st.markdown("""
    <style>
    .stApp {
        background: linear-gradient(to right, #BADFCD, #F1EDEA);
    }
    .block-container {
        background: transparent !important;
        padding: 2rem 3rem;
    }
    div[data-testid="stTabs"] {
        background-color: transparent !important;
        padding: 10px 0;
        border-bottom: 2px solid #e2e8f0;
    }
    div[data-testid="stTab"] {
        font-family: 'Segoe UI', sans-serif;
        font-size: 18px;
        font-weight: 600;
        color: #1e293b;
        background-color: #e2e8f0;
        border-radius: 12px 12px 0 0;
        margin-right: 12px;
        padding: 10px 24px;
        transition: all 0.3s ease;
        box-shadow: 0 2px 4px rgba(0,0,0,0.05);
    }
    div[data-testid="stTab"]:hover {
        background-color: #cbd5e1;
        color: #0f172a;
    }
    div[data-testid="stTab"][aria-selected="true"] {
        background-color: #3b82f6;
        color: white;
        box-shadow: 0 4px 6px rgba(0,0,0,0.1);
    }
    .stTextInput > div > div > input,
    .stNumberInput > div > div > input,
    .stSelectbox > div > div,
    .stRadio > div,
    .stTextArea > div > textarea {
        background-color: #f8fafc !important;
        border-radius: 10px !important;
    }
    h1, h2, h3 {
        color: #1e3a8a;
        font-family: 'Segoe UI', sans-serif;
    }
    p, li {
        color: #334155;
        font-size: 16px;
    }
    </style>
""", unsafe_allow_html=True)

# -------------------------------------------------------
# LOAD MODEL AND SCALER
# -------------------------------------------------------
@st.cache_resource
def load_artifacts():
    if not os.path.exists("thyroid_model.pkl"):
        st.error("Model file ('thyroid_model.pkl') is missing.")
        st.stop()
    if not os.path.exists("scaler.pkl"):
        st.warning("Scaler file ('scaler.pkl') not found. Proceeding without scaling.")
        scaler = None
    else:
        scaler = joblib.load("scaler.pkl")

    model = joblib.load("thyroid_model.pkl")
    label_map = {0: "No Thyroid Disorder", 1: "Hypothyroid", 2: "Hyperthyroid"}
    return model, scaler, label_map

model, scaler, label_map = load_artifacts()

# -------------------------------------------------------
# TOPBAR NAVIGATION
# -------------------------------------------------------
tabs = st.tabs(["Home", "Single-Patient Thyroid Analysis", "Multi-Patient Thyroid Analysis", "About REMI", "Learn More", "Wellness Hub", "Contact"])

st.markdown("""
    <div style='text-align:right; font-size:14px; margin-top:-10px;'>
        Prefer dark mode? Go to <b>Settings → Theme</b> and select <i>Dark</i>.
    </div>
""", unsafe_allow_html=True)


# -------------------------------------------------------
# HOME PAGE
# -------------------------------------------------------
with tabs[0]:
    # --- Hero Section ---
    st.markdown("""
        <div style='text-align: center; padding: 30px 0 10px 0;'>
            <h1 style='font-size: 42px; color: #228B22; margin-bottom: 10px;'>REMI – The Thyroid Disease Detector</h1>
            <p style='font-size:18px;'>
        <span style='color:#2E8B57;'>
                Empowering early thyroid detection through AI-driven insights.
                </span>
            </p>
        </div>
    """, unsafe_allow_html=True)

    st.markdown("<hr style='margin:25px 0;'>", unsafe_allow_html=True)

    # --- About REMI ---
    st.markdown("""
        <h2 style='color:#32CD32;'>About REMI</h2>
        <p style='font-size:16px; color:#334155;'>
        REMI (Rapid Endocrine Monitoring Interface) is an intelligent
        web-based application designed to assist in the early identification and
        assessment of thyroid disorders using machine learning models.
By analyzing key biochemical parameters from routine blood tests,
REMI classifies patient data as <b>normal</b>, <b>hypothyroid</b>, or <b>hyperthyroid</b>,
helping clinicians and users gain quick, data-based insights.

The tool provides an accessible platform for preliminary evaluation and
awareness, supporting faster decisions and encouraging timely medical consultation when abnormalities are detected. </p>
    """, unsafe_allow_html=True)

    # --- About Thyroid Section ---
    st.markdown("""
        <h2 style='color:#1e3a8a; margin-top:30px;'>Thyroid at a glance</h2>
        <p style='font-size:16px; color:#334155;'>

          The thyroid gland is a small, butterfly-shaped organ located at the base of the neck. It produces
            hormones such as T3 and T4, which regulate metabolism, growth, and energy levels throughout the body.
            An imbalance in thyroid hormone levels can lead to conditions like <b>hypothyroidism</b> (underactive thyroid)
            or <b>hyperthyroidism</b> (overactive thyroid), each with distinct clinical symptoms.
            Early detection through diagnostic indicators like TSH, TT4, and FTI can significantly improve health outcomes.
        </p>
    """, unsafe_allow_html=True)

    # --- Early Detection Section ---
    st.markdown("""
        <h2 style='color:#1e3a8a; margin-top:20px;'>Why Early Detection Matters</h2>
        <p style='font-size:16px; color:#334155;'>
            Timely screening of thyroid levels helps prevent complications such as fatigue, weight changes, and mood imbalance.
            AI-powered tools like REMI make it easier to spot warning signs early, empowering individuals to seek medical guidance sooner.
        </p>
    """, unsafe_allow_html=True)


    # --- Features Section ---
    st.markdown("""
        <h2 style='color:#1e3a8a; margin-top:30px;'>Core Features</h2>
        <div style='display:flex; flex-wrap:wrap; justify-content:space-around; margin-top:15px;'>
            <div style='background-color:#F8FAFC; padding:20px; border-radius:12px; width:45%; margin:10px; box-shadow:0 2px 6px rgba(0,0,0,0.1);'>
                <h3 style='color:#1e40af;'>Single Prediction</h3>
                <p>Input one patient’s lab test data to instantly predict thyroid condition and confidence level.</p>
            </div>
            <div style='background-color:#F8FAFC; padding:20px; border-radius:12px; width:45%; margin:10px; box-shadow:0 2px 6px rgba(0,0,0,0.1);'>
                <h3 style='color:#1e40af;'>Batch Prediction</h3>
                <p>Upload a CSV file containing multiple patient records to generate predictions in bulk.</p>
            </div>
            <div style='background-color:#F8FAFC; padding:20px; border-radius:12px; width:45%; margin:10px; box-shadow:0 2px 6px rgba(0,0,0,0.1);'>
                <h3 style='color:#1e40af;'>Confidence Insights</h3>
                <p>Each prediction includes a confidence score based on model probability values.</p>
            </div>
            <div style='background-color:#F8FAFC; padding:20px; border-radius:12px; width:45%; margin:10px; box-shadow:0 2px 6px rgba(0,0,0,0.1);'>
                <h3 style='color:#1e40af;'>User-Friendly Design</h3>
                <p>A clean, intuitive interface that makes clinical AI integration simple and efficient.</p>
            </div>
        </div>
    """, unsafe_allow_html=True)

    # --- Get Started Section ---
    st.markdown("""
        <div style='text-align:center; margin-top:40px;'>
            <p style='color:#475569; font-size:14px; margin-top:8px;'>
                Use the tabs above to navigate to Single or Batch Prediction.
            </p>
        </div>
    """, unsafe_allow_html=True)

    st.success("Model and Scaler loaded successfully!")



# -------------------------------------------------------
# SINGLE PREDICTION PAGE
# -------------------------------------------------------
with tabs[1]:
        st.markdown("<h1 style='text-align:center; color:#228B22; margin-bottom:10px;'>Single-Patient Thyroid Analysis </h1>", unsafe_allow_html=True)
        st.markdown("<hr style='border:1px solid #ddd'>", unsafe_allow_html=True)
        st.markdown("<p>Enter the details below as per the patient’s lab results.</p>", unsafe_allow_html=True)

        # --- Numeric inputs ---
        st.markdown("<h3 style='text-align:center;'>Lab Results & Basic Info</h3>", unsafe_allow_html=True)

        age = st.number_input("Age", 0, 120, 35)
        sex = st.selectbox("Sex", ["Female", "Male"])
        sex = 1 if sex == "Male" else 0

        TSH = st.number_input("TSH", 0.0, 500.0, 2.5)
        T3 = st.number_input("T3", 0.0, 10.0, 3.0)
        TT4 = st.number_input("TT4", 0.0, 500.0, 110.0)
        T4U = st.number_input("T4U", 0.0, 10.0, 1.0)
        FTI = st.number_input("FTI", 0.0, 500.0, 120.0)

            # --- Medical History ---
        st.markdown("<h3 style='text-align:center;'>Medical History</h3>", unsafe_allow_html=True)

        def yn(label):
            return st.radio(label, ["No", "Yes"], horizontal=True, key=label)

        # Use two evenly spaced columns across the full width
        col1, col2 = st.columns([1, 1], gap="large")

        with col1:
            on_thyroxine = 1 if yn("On Thyroxine") == "Yes" else 0
            query_on_thyroxine = 1 if yn("Query Thyroxine") == "Yes" else 0
            on_antithyroid_meds = 1 if yn("On Anti-thyroid Meds") == "Yes" else 0
            sick = 1 if yn("Recently Ill") == "Yes" else 0
            pregnant = 1 if yn("Pregnant") == "Yes" else 0
            thyroid_surgery = 1 if yn("Thyroid Surgery") == "Yes" else 0
            I131_treatment = 1 if yn("I131 Treatment") == "Yes" else 0

        with col2:

            query_hypothyroid = 1 if yn("Query Hypothyroid") == "Yes" else 0
            query_hyperthyroid = 1 if yn("Query Hyperthyroid") == "Yes" else 0
            lithium = 1 if yn("Lithium") == "Yes" else 0
            goitre = 1 if yn("Goitre") == "Yes" else 0
            tumor = 1 if yn("Tumor") == "Yes" else 0
            hypopituitary = 1 if yn("Hypopituitary") == "Yes" else 0
            psych = 1 if yn("Psychiatric History") == "Yes" else 0



        # --- Prepare input data ---
        input_data = {
            "age": age, "sex": sex, "on_thyroxine": on_thyroxine, "query_on_thyroxine": query_on_thyroxine,
            "on_antithyroid_meds": on_antithyroid_meds, "sick": sick, "pregnant": pregnant,
            "thyroid_surgery": thyroid_surgery, "I131_treatment": I131_treatment,
            "query_hypothyroid": query_hypothyroid, "query_hyperthyroid": query_hyperthyroid,
            "lithium": lithium, "goitre": goitre, "tumor": tumor, "hypopituitary": hypopituitary,
            "psych": psych, "TSH": TSH, "T3": T3, "TT4": TT4, "T4U": T4U, "FTI": FTI
        }

        input_df = pd.DataFrame([input_data])
        if hasattr(model, "feature_names_in_"):
            for col in model.feature_names_in_:
                if col not in input_df.columns:
                    input_df[col] = 0
            input_df = input_df[model.feature_names_in_]

        # --- Prediction button ---
        if st.button("Predict Result", type="primary"):
            try:
                if scaler is not None:
                    input_df[input_df.columns] = scaler.transform(input_df)
                pred = model.predict(input_df)[0]
                prob = model.predict_proba(input_df)[0]
                confidence = round(max(prob) * 100, 2)

                st.success(f"### Prediction: {label_map[pred]}")
                st.info(f"Confidence Level: **{confidence}%**")

                prob_df = pd.DataFrame({
                    "Condition": [label_map[i] for i in range(len(prob))],
                    "Probability (%)": [round(p * 100, 2) for p in prob]
                })
                st.table(prob_df)

            except Exception as e:
                st.error(f"Error during prediction: {e}")


# -------------------------------------------------------
# BATCH PREDICTION PAGE
# -------------------------------------------------------
with tabs[2]:
    st.markdown("<h1 style='text-align:center; color:#228B22; margin-bottom:10px;'>Multi-Patient Thyroid Analysis</h1>", unsafe_allow_html=True)
    st.markdown("<hr style='border:1px solid #ddd'>", unsafe_allow_html=True)

    uploaded_file = st.file_uploader("📤 Upload CSV File", type=["csv"])

    if uploaded_file is not None:
        try:
            df = pd.read_csv(uploaded_file)

            # ✅ FIX: Drop target column if present
            if "target" in df.columns:
                df = df.drop(columns=["target"])

            st.markdown("<h4 style='margin-top:20px;'> Data Preview</h4>", unsafe_allow_html=True)
            st.dataframe(df.head())

            if hasattr(model, "feature_names_in_"):
                missing_cols = [col for col in model.feature_names_in_ if col not in df.columns]
                if missing_cols:
                    st.warning(f"The following columns are missing and will be filled with zeros: {missing_cols}")
                    for col in missing_cols:
                        df[col] = 0
                df = df[model.feature_names_in_]

            if scaler is not None:
                df[df.columns] = scaler.transform(df)

            if st.button(" Run Batch Prediction"):
                preds = model.predict(df)
                probs = model.predict_proba(df)
                labels = [label_map[p] for p in preds]
                confidence = [f"{round(max(prob)*100,2)}%" for prob in probs]

                results_df = df.copy()
                results_df["Prediction"] = labels
                results_df["Confidence"] = confidence

                st.success("✅ Batch prediction completed successfully!")
                st.markdown("<h4>🧾 Prediction Results</h4>", unsafe_allow_html=True)
                st.dataframe(results_df)

                csv = results_df.to_csv(index=False).encode('utf-8')
                st.download_button(
                    label="📥 Download Results as CSV",
                    data=csv,
                    file_name="thyroid_batch_predictions.csv",
                    mime="text/csv",
                )
        except Exception as e:
            st.error(f"Error processing file: {e}")
    else:
        st.info("Please upload a CSV file to begin batch prediction.")

# -------------------------------------------------------
# ABOUT PAGE
# -------------------------------------------------------
with tabs[3]:
    st.markdown("<h1 style='text-align:center; color:#228B22; margin-bottom:10px;'>About This Application</h1>", unsafe_allow_html=True)
    st.markdown("<hr style='border:1px solid #ddd'>", unsafe_allow_html=True)

    st.markdown("""
    The Thyroid Disease Detection System is a machine learning-driven web application
    developed to assist in the identification and preliminary assessment of thyroid disorders.
    By analyzing key biochemical indicators from standard blood tests, the system provides
    insights into whether an individual is likely to have hypothyroidism, **hyperthyroidism,
    or normal thyroid function.

    This solution reflects the integration of artificial intelligence into clinical analytics.
    It leverages data-driven modeling to support healthcare professionals, laboratories,
    and research institutions in making informed evaluations.
    """, unsafe_allow_html=True)

    st.markdown("<h2>System Overview</h2>", unsafe_allow_html=True )
    st.markdown("""
    The platform processes essential parameters including TSH, T3, TT4, T4U, and FTI,
    along with patient information such as age, sex, and relevant medical factors.
    Using a trained predictive model, it classifies patient data into specific diagnostic categories.

    The application provides two key functionalities:
    - Single Prediction: Input patient values manually to receive an immediate diagnostic prediction.
    - Batch Prediction: Upload a CSV file to generate predictions for multiple records simultaneously.
    Each output includes the predicted condition and a confidence score derived from model probabilities.
    """, unsafe_allow_html=True)

    st.markdown("<h2>Core Technologies</h2>", unsafe_allow_html=True)
    st.markdown("""
    - Python & Streamlit for interactive web application development
    - Scikit-learn for machine learning modeling and evaluation
    - Pandas & NumPy for data analysis and transformation
    - Joblib for efficient model persistence and deployment
    """, unsafe_allow_html=True)

    st.markdown("<h2>Design Philosophy and Objective</h2>", unsafe_allow_html=True)
    st.markdown("""
    The design of this system focuses on accuracy, interpretability, and simplicity of use.
    In healthcare applications, reliable predictions and user clarity are essential.
    This system aims to demonstrate how predictive analytics can complement
    traditional laboratory diagnostics by providing quick, data-informed insights.

    The tool is intended for research, education, and clinical support environments
    where AI-assisted decision systems can improve early detection and diagnostic efficiency.
    It is not intended to replace professional medical evaluation or laboratory analysis.
    """, unsafe_allow_html=True)



# -------------------------------------------------------
# LEARN PAGE
# -------------------------------------------------------
with tabs[4]:
    st.markdown("<h1 style='text-align:center; color:#228B22; margin-bottom:10px;'>About REMI - The Thyroid Disease Detector</h1>", unsafe_allow_html=True)
    st.markdown("<hr style='border:1px solid #ddd'>", unsafe_allow_html=True)
    # custom styling for headings, text color, and boxes
    st.markdown("""
        <style>
        h2, h3, h4 {
            color: #015A84 !important;
        }
        body, p, li, td, th {
            color: #1E293B !important;  /* deep gray-blue text */
        }
        .info-box {
            border: 1px solid #000;
            border-radius: 10px;
            background-color: rgba(255, 255, 255, 0.6);
            padding: 15px;
            margin-top: 10px;
            margin-bottom: 20px;
        }
        table {
            width: 100%;
            border-collapse: collapse;
            margin-top: 10px;
            color: #1E293B;
        }
        th, td {
            border: 1px solid #000;
            padding: 8px;
            text-align: left;
        }
        th {
            background-color: rgba(1, 90, 132, 0.1);
            color: #015A84;
        }
        ul {
            color: #1E293B;
        }
        </style>
    """, unsafe_allow_html=True)

    st.markdown("<h2>Learn About Thyroid Disorders</h2>", unsafe_allow_html=True)
    st.markdown("""
    The thyroid gland plays a vital role in regulating metabolism, energy, and hormone balance.
    Understanding thyroid disorders can help identify symptoms early and improve management.

    ### Common Thyroid Conditions
    - Hypothyroidism: The thyroid produces too little hormone.
        Symptoms: fatigue, weight gain, cold intolerance, depression.
    - Hyperthyroidism: The thyroid produces too much hormone.
        Symptoms: weight loss, anxiety, tremors, heat intolerance.
    - Goitre: Enlargement of the thyroid gland, can occur with or without hormone imbalance.
    """, unsafe_allow_html=True)

    # Common Symptoms (no box)
    st.markdown("""
    <h3>Common Symptoms of Thyroid Imbalance</h3>
    <table>
    <tr><th>Type</th><th>Symptoms</th></tr>
    <tr><td>Hypothyroidism</td><td>Fatigue, dry skin, weight gain, hair loss, feeling cold, slow heart rate</td></tr>
    <tr><td>Hyperthyroidism</td><td>Weight loss, irritability, sweating, rapid heartbeat, trembling hands</td></tr>
    </table>
    """, unsafe_allow_html=True)

    # Key Blood Tests box
    st.markdown("""
    <div class="info-box">
    <h3>Key Blood Tests</h3>
    <table>
        <tr>
            <th>Test</th><th>Measures</th><th>Typical Range</th><th>Meaning</th>
        </tr>
        <tr>
            <td><i>TSH</i></td><td>Thyroid Stimulating Hormone</td><td>0.4 – 4.0 mIU/L</td><td>High = Hypothyroid; Low = Hyperthyroid</td>
        </tr>
        <tr>
            <td><i>T3</i></td><td>Triiodothyronine</td><td>0.8 – 2.0 ng/mL</td><td>Active thyroid hormone</td>
        </tr>
        <tr>
            <td><i>TT4</i></td><td>Total Thyroxine</td><td>60 – 150 µg/dL</td><td>Total circulating T4</td>
        </tr>
        <tr>
            <td><i>T4U</i></td><td>Thyroxine Uptake</td><td>0.7 – 1.8</td><td>Measures T4-binding capacity</td>
        </tr>
        <tr>
            <td><i>FTI</i></td><td>Free Thyroxine Index</td><td>80 – 180</td><td>Estimates free thyroid hormone levels</td>
        </tr>
    </table>
    </div>
    """, unsafe_allow_html=True)

    # Thyroid diet box
    st.markdown("""
    <div class="info-box">
    <h3>Healthy Diet for Thyroid Health (Indian Menu)</h3>
    A balanced diet supports thyroid function and helps manage symptoms effectively.
    Focus on nutrient-rich foods and avoid those that interfere with hormone absorption.

    <h4>What to Include</h4>
    <table>
        <tr><th>Category</th><th>Examples</th></tr>
        <tr><td>Iodine-rich foods</td><td>Iodized salt, seaweed (moderation), curd</td></tr>
        <tr><td>Protein sources</td><td>Moong dal, lentils, paneer, eggs, chicken, fish</td></tr>
        <tr><td>Selenium & Zinc</td><td>Sunflower seeds, pumpkin seeds, cashews, whole grains</td></tr>
        <tr><td>Fibre & Antioxidants</td><td>Fruits (papaya, apple, pomegranate), vegetables (palak, lauki, beans)</td></tr>
        <tr><td>Healthy fats</td><td>Ghee (in moderation), olive oil, nuts, seeds</td></tr>
    </table>

    <h4>Foods to Limit or Avoid</h4>
    <table>
        <tr><th>Type</th><th>Examples</th></tr>
        <tr><td>Goitrogenic foods (excess)</td><td>Raw cabbage, cauliflower, broccoli (cook before eating)</td></tr>
        <tr><td>Soy products</td><td>Soy chunks, tofu, soy milk</td></tr>
        <tr><td>Refined sugar & carbs</td><td>Pastries, sweets, white bread</td></tr>
        <tr><td>Caffeine & alcohol</td><td>Limit intake to avoid absorption issues</td></tr>
    </table>

    <h4>Sample One-Day Thyroid-Friendly Meal Plan</h4>
    <table>
        <tr><th>Time</th><th>Meal</th></tr>
        <tr><td>Morning (empty stomach)</td><td>Thyroid medication + lukewarm water</td></tr>
        <tr><td>Breakfast</td><td>Vegetable oats upma + 1 boiled egg / glass of milk</td></tr>
        <tr><td>Mid-morning</td><td>Fruit bowl (papaya or apple)</td></tr>
        <tr><td>Lunch</td><td>Brown rice / phulka + dal + palak sabzi + curd</td></tr>
        <tr><td>Evening</td><td>Roasted chana / handful of nuts</td></tr>
        <tr><td>Dinner</td><td>Moong dal khichdi / grilled paneer + sautéed veggies</td></tr>
        <tr><td>Before bed</td><td>Warm turmeric milk (optional)</td></tr>
    </table>
    </div>
    """, unsafe_allow_html=True)

    # Everyday healthy diet box
    st.markdown("""
    <div class="info-box">
    <h3>Everyday Healthy Diet (Indian Style)</h3>
    <p>Maintaining a balanced diet helps support energy, immunity, and long-term wellness.</p>

    <h4>Basic Guidelines</h4>
    <ul>
        <li>Eat fresh, home-cooked meals with balanced portions.</li>
        <li>Include all food groups: whole grains, proteins, healthy fats, fruits, and vegetables.</li>
        <li>Avoid overeating, processed snacks, and sugary drinks.</li>
        <li>Stay hydrated and active throughout the day.</li>
    </ul>

    <h4>Ideal Daily Indian Meal Plan</h4>
    <table>
        <tr><th>Time</th><th>Meal</th></tr>
        <tr><td>Early Morning</td><td>Warm water with lemon / soaked almonds (4–5)</td></tr>
        <tr><td>Breakfast</td><td>Poha / vegetable paratha with curd / idli with sambar</td></tr>
        <tr><td>Mid-Morning</td><td>Seasonal fruit (banana, apple, orange)</td></tr>
        <tr><td>Lunch</td><td>Brown rice / 2 rotis + dal + sabzi + salad + curd</td></tr>
        <tr><td>Evening Snack</td><td>Sprouts chaat / roasted makhana / green tea</td></tr>
        <tr><td>Dinner</td><td>Light meal – khichdi / vegetable soup + grilled paneer or tofu</td></tr>
        <tr><td>Before Bed</td><td>Warm milk with turmeric (optional)</td></tr>
    </table>

    <h4>Healthy Habits</h4>
    <ul>
        <li>Drink 8–10 glasses of water daily.</li>
        <li>Limit fried food, sugar, and packaged juices.</li>
        <li>Practice mindful eating — eat slowly and avoid screens.</li>
        <li>Get 7–8 hours of quality sleep.</li>
    </ul>
    </div>
    """, unsafe_allow_html=True)

    # Quick Do’s & Don’ts (no box)
    st.markdown("""
    <h3>Quick Do’s & Don’ts</h3>
    <table>
    <tr><th>Do</th><th>Don’t</th></tr>
    <tr><td>Take thyroid medication on an empty stomach</td><td>Skip doses or take with coffee/milk</td></tr>
    <tr><td>Eat iodine and selenium-rich foods</td><td>Overconsume raw cruciferous veggies</td></tr>
    <tr><td>Exercise moderately</td><td>Ignore fatigue or sudden mood swings</td></tr>
    </table>
    """, unsafe_allow_html=True)

    # Disclaimer
    st.markdown("""
    ### Medical Disclaimer
    This app is for educational and research purposes only.
    It should not be used for self-diagnosis or treatment decisions.
    Always consult a licensed healthcare provider for clinical interpretation.
    """, unsafe_allow_html=True)



# -------------------------------------------------------
# WELLNESS HUB PAGE
# -------------------------------------------------------
with tabs[5]:
    st.markdown("<h1 style='text-align:center; color:#228B22; margin-bottom:10px;'>Wellness Hub – Find Trusted Thyroid Specialists in India</h1>", unsafe_allow_html=True)
    st.markdown("<hr style='border:1px solid #ddd'>", unsafe_allow_html=True)
    st.markdown("""
        <p style='font-size:16px; color:#475569;'>
            Discover top hospitals and endocrinology specialists across India specializing in thyroid care and hormonal health.
            Use this directory to find reliable medical consultation and advanced diagnostic facilities.
        </p>
    """, unsafe_allow_html=True)

    hospitals = [
        {
            "city": "New Delhi",
            "hospital": "All India Institute of Medical Sciences (AIIMS)",
            "doctor": "Dr. Ramesh Goyal – Endocrinologist",
            "address": "Ansari Nagar, New Delhi – 110029",
            "contact": "+91 11 2658 8500"
        },
        {
            "city": "Mumbai",
            "hospital": "Fortis Hospital, Mulund",
            "doctor": "Dr. Shashank Joshi – Senior Consultant, Endocrinology",
            "address": "Mulund Goregaon Link Rd, Mumbai – 400078",
            "contact": "+91 22 6799 3333"
        },
        {
            "city": "Chennai",
            "hospital": "Apollo Hospital",
            "doctor": "Dr. V. Mohan – Chief Diabetologist & Endocrinologist",
            "address": "Greams Lane, Off Greams Road, Chennai – 600006",
            "contact": "+91 44 2829 3333"
        },
        {
            "city": "Bengaluru",
            "hospital": "Manipal Hospital",
            "doctor": "Dr. Suresh Raghavan – Consultant Endocrinologist",
            "address": "Old Airport Road, Bengaluru – 560017",
            "contact": "+91 80 2222 1111"
        },
        {
            "city": "Kolkata",
            "hospital": "AMRI Hospitals",
            "doctor": "Dr. Anirban Majumder – Endocrinologist",
            "address": "JC-16 & 17, Salt Lake City, Kolkata – 700098",
            "contact": "+91 33 6680 0000"
        },
        {
            "city": "Hyderabad",
            "hospital": "Yashoda Hospitals",
            "doctor": "Dr. Nitin Kapoor – Consultant Endocrinologist",
            "address": "Raj Bhavan Road, Somajiguda, Hyderabad – 500082",
            "contact": "+91 40 4567 4567"
        },
        {
            "city": "Pune",
            "hospital": "Ruby Hall Clinic",
            "doctor": "Dr. Rajesh Rajput – Thyroid & Hormone Specialist",
            "address": "40, Sassoon Road, Pune – 411001",
            "contact": "+91 20 2616 3391"
        },
        {
            "city": "Ahmedabad",
            "hospital": "Sterling Hospital",
            "doctor": "Dr. Harshal Joshi – Consultant Endocrinologist",
            "address": "Gurukul Road, Memnagar, Ahmedabad – 380052",
            "contact": "+91 79 4001 1111"
        }
    ]

    # Style the cards
    st.markdown("""
        <style>
        .hospital-card {
            background-color: #F1EDEA;
            border-radius: 12px;
            padding: 18px 22px;
            margin-bottom: 16px;
            box-shadow: 0 2px 6px rgba(0,0,0,0.08);
            border: 1px solid #e2e8f0;
        }
        .hospital-card h4 {
            margin: 0;
            color: #1e3a8a;
        }
        .hospital-card p {
            margin: 4px 0;
            color: #334155;
        }
        </style>
    """, unsafe_allow_html=True)

    for entry in hospitals:
        st.markdown(f"""
            <div class='hospital-card'>
                <h4>{entry['hospital']} ({entry['city']})</h4>
                <p><b>Specialist:</b> {entry['doctor']}</p>
                <p><b>Address:</b> {entry['address']}</p>
                <p><b>Contact:</b> {entry['contact']}</p>
            </div>
        """, unsafe_allow_html=True)


# -------------------------------------------------------
# CONTACT PAGE
# -------------------------------------------------------
with tabs[6]:
    st.markdown("<h1 style='text-align:center; color:#228B22; margin-bottom:10px;'>Have Questions? Contact Us!</h1>", unsafe_allow_html=True)
    st.markdown("<hr style='border:1px solid #ddd'>", unsafe_allow_html=True)
    st.markdown("""
        <style>
        /* ===== Input fields (Your Name, Your Email) ===== */
        div[data-baseweb="input"] {
            background-color: #F1EDEA !important;
            border-radius: 10px !important;
            border: 1px solid #d1d5db !important;
            box-shadow: 0px 2px 4px rgba(0,0,0,0.05) !important;
        }
        div[data-baseweb="input"] input {
            background-color: #F1EDEA !important;
            color: #1e293b !important;
            border: none !important;
        }

        /* ===== Text area (Your Message) ===== */
        div[data-baseweb="textarea"] {
            background-color: #ffffff !important;   /* white box look */
            border-radius: 10px !important;
            border: 1px solid #d1d5db !important;
            box-shadow: 0px 2px 6px rgba(0,0,0,0.08) !important;
        }
        div[data-baseweb="textarea"] textarea {
            background-color: #ffffff !important;
            color: #1e293b !important;
            border: none !important;
            border-radius: 10px !important;
            padding: 10px !important;
        }

        /* ===== Button styling ===== */
        .stForm button[kind="primary"] {
            background-color: #F1EDEA !important;
            color: #1e293b !important;
            border: 1px solid #cbd5e1 !important;
            border-radius: 10px !important;
            transition: all 0.2s ease-in-out !important;
        }
        .stForm button[kind="primary"]:hover {
            background-color: #e7deda !important;
            color: #111827 !important;
            border-color: #94a3b8 !important;
        }

        /* ===== Header and list styling ===== */
        h2, h3 { color: #1e293b !important; }

        .team-list {
            list-style-type: none;
            padding-left: 0;
        }
        .team-list li {
            background-color: #F1EDEA;
            padding: 12px 18px;
            margin-bottom: 10px;
            border-radius: 10px;
            font-size: 16px;
            color: #1e293b;
            display: flex;
            justify-content: space-between;
            align-items: center;
            border: 1px solid #d1d5db;
            box-shadow: 0 2px 4px rgba(0,0,0,0.05);
        }

        /* ===== LinkedIn button styling ===== */
        .linkedin-btn {
            background-color: #0077B5;
            color: white !important;
            text-decoration: none;
            padding: 6px 12px;
            border-radius: 6px;
            font-size: 14px;
            font-weight: 500;
            transition: all 0.2s ease-in-out;
        }
        .linkedin-btn:hover {
            background-color: #005b86;
        }
        </style>
    """, unsafe_allow_html=True)

    # --- Contact form section ---
    st.markdown("<h2>Contact Us</h2>", unsafe_allow_html=True)
    with st.form("contact_form"):
        name = st.text_input("Your Name")
        email = st.text_input("Your Email")
        message = st.text_area("Your Message")
        submitted = st.form_submit_button("Send Message")
        if submitted:
            st.success("✅ Thank you! Your message has been received.")

    # --- Team section with LinkedIn buttons ---
    st.markdown("""
        <h3>Connect With Us</h3>
        <ul class="team-list">
            <li><span><b>Ishika Das</b> – 2205289</span>
                <a class="linkedin-btn" href="https://www.linkedin.com/in/ishika-das-39aa2324a/" target="_blank">LinkedIn</a>
            </li>
            <li><span><b>Anishka</b> – 2205356</span>
                <a class="linkedin-btn" href="https://www.linkedin.com/in/anishka2011/" target="_blank">LinkedIn</a>
            </li>
            <li><span><b>Sangbarta Sarkar</b> – 22051612</span>
                <a class="linkedin-btn" href="https://www.linkedin.com/in/sangbartasarkar02" target="_blank">LinkedIn</a>
            </li>
            <li><span><b>Aryan Pattnaik</b> – 2228018</span>
                <a class="linkedin-btn" href="https://www.linkedin.com/in/aryan-pattnaik/?originalSubdomain=in" target="_blank">LinkedIn</a>
            </li>
        </ul>
    """, unsafe_allow_html=True)


# --- Footer ---
st.markdown("""
    <hr style='margin-top:50px;'>
    <div style='text-align: center; font-size: 13px; color: #64748b;'>
        © 2025 <b>REMI</b> | Powered by Streamlit and Machine Learning
    </div>
""", unsafe_allow_html=True)

Overwriting thyroid_app.py


In [ ]:
%%writefile thyroid_app.py
import streamlit as st
import pandas as pd
import numpy as np
import joblib
import os

# -------------------------------------------------------
# PAGE CONFIGURATION
# -------------------------------------------------------
st.set_page_config(page_title="Thyroid Disease Detector", layout="wide")


# -------------------------------------------------------
# CUSTOM STYLING
# -------------------------------------------------------
st.markdown("""
    <style>
    .stApp {
        background: linear-gradient(to right, #BADFCD, #F1EDEA);
    }
    .block-container {
        background: transparent !important;
        padding: 2rem 3rem;
    }
    div[data-testid="stTabs"] {
        background-color: transparent !important;
        padding: 10px 0;
        border-bottom: 2px solid #e2e8f0;
    }
    div[data-testid="stTab"] {
        font-family: 'Segoe UI', sans-serif;
        font-size: 18px;
        font-weight: 600;
        color: #1e293b;
        background-color: #e2e8f0;
        border-radius: 12px 12px 0 0;
        margin-right: 12px;
        padding: 10px 24px;
        transition: all 0.3s ease;
        box-shadow: 0 2px 4px rgba(0,0,0,0.05);
    }
    div[data-testid="stTab"]:hover {
        background-color: #cbd5e1;
        color: #0f172a;
    }
    div[data-testid="stTab"][aria-selected="true"] {
        background-color: #3b82f6;
        color: white;
        box-shadow: 0 4px 6px rgba(0,0,0,0.1);
    }
    .stTextInput > div > div > input,
    .stNumberInput > div > div > input,
    .stSelectbox > div > div,
    .stRadio > div,
    .stTextArea > div > textarea {
        background-color: #f8fafc !important;
        border-radius: 10px !important;
    }
    h1, h2, h3 {
        color: #1e3a8a;
        font-family: 'Segoe UI', sans-serif;
    }
    p, li {
        color: #334155;
        font-size: 16px;
    }
    </style>
""", unsafe_allow_html=True)

# -------------------------------------------------------
# LOAD MODEL AND SCALER
# -------------------------------------------------------
@st.cache_resource
def load_artifacts():
    if not os.path.exists("thyroid_model.pkl"):
        st.error("Model file ('thyroid_model.pkl') is missing.")
        st.stop()
    if not os.path.exists("scaler.pkl"):
        st.warning("Scaler file ('scaler.pkl') not found. Proceeding without scaling.")
        scaler = None
    else:
        scaler = joblib.load("scaler.pkl")

    model = joblib.load("thyroid_model.pkl")
    label_map = {0: "No Thyroid Disorder", 1: "Hypothyroid", 2: "Hyperthyroid"}
    return model, scaler, label_map

model, scaler, label_map = load_artifacts()

# -------------------------------------------------------
# TOPBAR NAVIGATION
# -------------------------------------------------------
tabs = st.tabs(["Home", "Single-Patient Thyroid Analysis", "Multi-Patient Thyroid Analysis", "About REMI", "Learn More", "Wellness Hub", "Contact"])

st.markdown("""
    <div style='text-align:right; font-size:14px; margin-top:-10px;'>
        Prefer dark mode? Go to <b>Settings → Theme</b> and select <i>Dark</i>.
    </div>
""", unsafe_allow_html=True)


# -------------------------------------------------------
# HOME PAGE
# -------------------------------------------------------
with tabs[0]:
    # --- Hero Section ---
    st.markdown("""
        <div style='text-align: center; padding: 30px 0 10px 0;'>
            <h1 style='font-size: 42px; color: #228B22; margin-bottom: 10px;'>REMI – The Thyroid Disease Detector</h1>
            <p style='font-size:18px;'>
        <span style='color:#2E8B57;'>
                Empowering early thyroid detection through AI-driven insights.
                </span>
            </p>
        </div>
    """, unsafe_allow_html=True)

    st.markdown("<hr style='margin:25px 0;'>", unsafe_allow_html=True)

    # --- About REMI ---
    st.markdown("""
        <p style='font-size:16px; color:#334155;'>
        REMI (Rapid Endocrine Monitoring Interface) is a structured analytical platform
        developed to enhance the interpretation of endocrine-related laboratory data through
        computational precision. It operates as an integrated system that transforms raw
        biochemical inputs into organized, meaningful insights using machine learning–based evaluation.

At its core, REMI is designed to manage and interpret complex laboratory parameters efficiently.
The system consolidates multiple data points, aligns them within a standardized analytical framework,
and generates a consistent interpretation that highlights potential physiological trends. This
structured processing allows REMI to transform scattered quantitative information into a coherent
analytical output that users can easily understand and act upon.

The development of REMI emphasizes clarity, reproducibility, and streamlined workflow.
Its architecture supports the systematic handling of input variables, automated processing,
and smooth delivery of results, reducing the cognitive load associated with manual evaluation.
Beyond computation, REMI prioritizes usability—providing an interface that is organized, responsive,
and accessible to users with varying levels of technical expertise.

REMI serves as a bridge between raw laboratory measurements and structured analytical insight.
It enables users to visualize biochemical information through a refined computational lens, offering
a more organized perspective on endocrine data than traditional manual interpretation. The system is
designed not only for accuracy, but also for consistency, ensuring that each evaluation follows a uniform analytical approach.
Through its combination of algorithmic evaluation, intuitive design, and efficient data handling,
REMI establishes itself as a dependable tool for environments where precision, clarity, and analytical depth are essential.
        </p>
    """, unsafe_allow_html=True)

    # --- About Thyroid Section ---
    st.markdown("""
        <h2 style='color:#1e3a8a; margin-top:30px;'>Thyroid at a glance</h2>
        <p style='font-size:16px; color:#334155;'>

          The thyroid gland is a small, butterfly-shaped organ located at the base of the neck. It produces
            hormones such as T3 and T4, which regulate metabolism, growth, and energy levels throughout the body.
            An imbalance in thyroid hormone levels can lead to conditions like <b>hypothyroidism</b> (underactive thyroid)
            or <b>hyperthyroidism</b> (overactive thyroid), each with distinct clinical symptoms.
            Early detection through diagnostic indicators like TSH, TT4, and FTI can significantly improve health outcomes.
        </p>
    """, unsafe_allow_html=True)

    # --- Early Detection Section ---
    st.markdown("""
        <h2 style='color:#1e3a8a; margin-top:20px;'>Why Early Detection Matters</h2>
        <p style='font-size:16px; color:#334155;'>
            Early detection of thyroid irregularities is essential for maintaining long-term metabolic, cognitive, and physiological balance.
            Thyroid dysfunction often begins with subtle changes that may go unnoticed—fatigue that is dismissed as routine stress,
            weight fluctuations attributed to lifestyle, or mood variations perceived as temporary. When these early signs are overlooked,
            the underlying hormonal imbalance can gradually progress, affecting multiple systems across the body.Timely screening enables individuals and clinicians to identify these deviations before they evolve into more
            pronounced or chronic conditions. Detecting abnormalities in their initial stages supports quicker intervention,
            more effective management, and a significant reduction in the likelihood of secondary complications.

Analytical systems such as REMI contribute to this process by transforming laboratory data into structured, evidence-aligned
insights. Instead of relying solely on manual interpretation, REMI streamlines the evaluation of biochemical markers,
highlights early patterns of concern, and enables users to recognize emerging trends with greater clarity.

By facilitating earlier recognition of potential endocrine disturbances, REMI encourages proactive health
monitoring and empowers individuals to seek professional assessment at the most impactful moment—when early action
can make a measurable difference in long-term outcomes.
        </p>
    """, unsafe_allow_html=True)


    # --- Features Section ---
    st.markdown("""
        <h2 style='color:#1e3a8a; margin-top:30px;'>Core Features</h2>
        <div style='display:flex; flex-wrap:wrap; justify-content:space-around; margin-top:15px;'>
            <div style='background-color:#F8FAFC; padding:20px; border-radius:12px; width:45%; margin:10px; box-shadow:0 2px 6px rgba(0,0,0,0.1);'>
                <h3 style='color:#1e40af;'>Single Prediction</h3>
                <p>Input one patient’s lab test data to instantly predict thyroid condition and confidence level.</p>
            </div>
            <div style='background-color:#F8FAFC; padding:20px; border-radius:12px; width:45%; margin:10px; box-shadow:0 2px 6px rgba(0,0,0,0.1);'>
                <h3 style='color:#1e40af;'>Batch Prediction</h3>
                <p>Upload a CSV file containing multiple patient records to generate predictions in bulk.</p>
            </div>
            <div style='background-color:#F8FAFC; padding:20px; border-radius:12px; width:45%; margin:10px; box-shadow:0 2px 6px rgba(0,0,0,0.1);'>
                <h3 style='color:#1e40af;'>Confidence Insights</h3>
                <p>Each prediction includes a confidence score based on model probability values.</p>
            </div>
            <div style='background-color:#F8FAFC; padding:20px; border-radius:12px; width:45%; margin:10px; box-shadow:0 2px 6px rgba(0,0,0,0.1);'>
                <h3 style='color:#1e40af;'>User-Friendly Design</h3>
                <p>A clean, intuitive interface that makes clinical AI integration simple and efficient.</p>
            </div>
        </div>
    """, unsafe_allow_html=True)

    # --- FAQs---
    st.markdown("""
        <h2 style='color:#1e3a8a; margin-top:20px;'>Frequently Asked Questions</h2>
        <p style='font-size:16px; color:#334155;'>
        <b>1. Can REMI replace my doctor?</b>

No. REMI is an analytical support system designed to assist in understanding laboratory data.
It provides structured, machine-generated insights, but it does not replace professional medical judgment,
clinical evaluation, or diagnostic procedures. All interpretations should be reviewed in consultation
with a qualified healthcare provider.

<b>2. What do confidence scores mean?</b>

Confidence scores represent the model’s estimated level of certainty in a particular classification.
A higher score indicates stronger alignment between the input data and the patterns learned by the
system during training. These scores help users understand how strongly the model favors one classification
over another, but they should not be viewed as absolute indicators of clinical accuracy.

<b>3. How does REMI analyze the data?</b>

REMI processes biochemical parameters through a trained machine learning model that identifies
statistical patterns associated with various endocrine conditions. The system evaluates inputs
against these learned patterns and produces a classification based on overall similarity.

<b>4. Who can use REMI?</b>

REMI is designed for students, researchers, and individuals seeking preliminary insight into laboratory
data. It is suitable for educational and analytical use but is not intended to be used as a substitute for medical expertise.

<b>5. Why should I use REMI for early screening?</b>

REMI provides structured, consistent interpretation of data, allowing users to observe potential
irregularities earlier than they might through manual review. This can support more timely follow-up
and informed discussions with healthcare professionals.
        </p>
    """, unsafe_allow_html=True)

    # --- Get Started Section ---
    st.markdown("""
        <div style='text-align:center; margin-top:40px;'>
            <p style='color:#475569; font-size:14px; margin-top:8px;'>
                Use the tabs above to navigate to Single or Batch Prediction.
            </p>
        </div>
    """, unsafe_allow_html=True)

    st.success("Model and Scaler loaded successfully!")



# -------------------------------------------------------
# SINGLE PREDICTION PAGE
# -------------------------------------------------------
    with tabs[1]:
        st.markdown("<h1 style='text-align:center; color:#228B22; margin-bottom:10px;'>Single-Patient Thyroid Analysis </h1>", unsafe_allow_html=True)
        st.markdown("<hr style='border:1px solid #ddd'>", unsafe_allow_html=True)
        st.markdown("<p>Enter the details below as per the patient’s lab results.</p>", unsafe_allow_html=True)

        # --- Numeric inputs ---
        st.markdown("<h3 style='text-align:center;'>Lab Results & Basic Info</h3>", unsafe_allow_html=True)

        age = st.number_input("Age", 0, 120, 35)
        sex = st.selectbox("Sex", ["Female", "Male"])
        sex = 1 if sex == "Male" else 0

        TSH = st.number_input("TSH", 0.0, 500.0, 2.5)
        T3 = st.number_input("T3", 0.0, 10.0, 3.0)
        TT4 = st.number_input("TT4", 0.0, 500.0, 110.0)
        T4U = st.number_input("T4U", 0.0, 10.0, 1.0)
        FTI = st.number_input("FTI", 0.0, 500.0, 120.0)

            # --- Medical History ---
        st.markdown("<h3 style='text-align:center;'>Medical History</h3>", unsafe_allow_html=True)

        def yn(label):
            return st.radio(label, ["No", "Yes"], horizontal=True, key=label)

        # Use two evenly spaced columns across the full width
        col1, col2 = st.columns([1, 1], gap="large")

        with col1:
            on_thyroxine = 1 if yn("On Thyroxine") == "Yes" else 0
            query_on_thyroxine = 1 if yn("Query Thyroxine") == "Yes" else 0
            on_antithyroid_meds = 1 if yn("On Anti-thyroid Meds") == "Yes" else 0
            sick = 1 if yn("Recently Ill") == "Yes" else 0
            pregnant = 1 if yn("Pregnant") == "Yes" else 0
            thyroid_surgery = 1 if yn("Thyroid Surgery") == "Yes" else 0
            I131_treatment = 1 if yn("I131 Treatment") == "Yes" else 0

        with col2:

            query_hypothyroid = 1 if yn("Query Hypothyroid") == "Yes" else 0
            query_hyperthyroid = 1 if yn("Query Hyperthyroid") == "Yes" else 0
            lithium = 1 if yn("Lithium") == "Yes" else 0
            goitre = 1 if yn("Goitre") == "Yes" else 0
            tumor = 1 if yn("Tumor") == "Yes" else 0
            hypopituitary = 1 if yn("Hypopituitary") == "Yes" else 0
            psych = 1 if yn("Psychiatric History") == "Yes" else 0



        # --- Prepare input data ---
        input_data = {
            "age": age, "sex": sex, "on_thyroxine": on_thyroxine, "query_on_thyroxine": query_on_thyroxine,
            "on_antithyroid_meds": on_antithyroid_meds, "sick": sick, "pregnant": pregnant,
            "thyroid_surgery": thyroid_surgery, "I131_treatment": I131_treatment,
            "query_hypothyroid": query_hypothyroid, "query_hyperthyroid": query_hyperthyroid,
            "lithium": lithium, "goitre": goitre, "tumor": tumor, "hypopituitary": hypopituitary,
            "psych": psych, "TSH": TSH, "T3": T3, "TT4": TT4, "T4U": T4U, "FTI": FTI
        }

        input_df = pd.DataFrame([input_data])
        if hasattr(model, "feature_names_in_"):
            for col in model.feature_names_in_:
                if col not in input_df.columns:
                    input_df[col] = 0
            input_df = input_df[model.feature_names_in_]

        # --- Prediction button ---
        if st.button("Predict Result", type="primary"):
            try:
                if scaler is not None:
                    input_df[input_df.columns] = scaler.transform(input_df)
                pred = model.predict(input_df)[0]
                prob = model.predict_proba(input_df)[0]
                confidence = round(max(prob) * 100, 2)

                st.success(f"### Prediction: {label_map[pred]}")
                st.info(f"Confidence Level: **{confidence}%**")

                prob_df = pd.DataFrame({
                    "Condition": [label_map[i] for i in range(len(prob))],
                    "Probability (%)": [round(p * 100, 2) for p in prob]
                })
                st.table(prob_df)

            except Exception as e:
                st.error(f"Error during prediction: {e}")


# -------------------------------------------------------
# BATCH PREDICTION PAGE
# -------------------------------------------------------
with tabs[2]:
    st.markdown("<h1 style='text-align:center; color:#228B22; margin-bottom:10px;'>Multi-Patient Thyroid Analysis</h1>", unsafe_allow_html=True)
    st.markdown("<hr style='border:1px solid #ddd'>", unsafe_allow_html=True)

    uploaded_file = st.file_uploader("📤 Upload CSV File", type=["csv"])

    if uploaded_file is not None:
        try:
            df = pd.read_csv(uploaded_file)

            # ✅ FIX: Drop target column if present
            if "target" in df.columns:
                df = df.drop(columns=["target"])

            st.markdown("<h4 style='margin-top:20px;'> Data Preview</h4>", unsafe_allow_html=True)
            st.dataframe(df.head())

            if hasattr(model, "feature_names_in_"):
                missing_cols = [col for col in model.feature_names_in_ if col not in df.columns]
                if missing_cols:
                    st.warning(f"The following columns are missing and will be filled with zeros: {missing_cols}")
                    for col in missing_cols:
                        df[col] = 0
                df = df[model.feature_names_in_]

            if scaler is not None:
                df[df.columns] = scaler.transform(df)

            if st.button(" Run Batch Prediction"):
                preds = model.predict(df)
                probs = model.predict_proba(df)
                labels = [label_map[p] for p in preds]
                confidence = [f"{round(max(prob)*100,2)}%" for prob in probs]

                results_df = df.copy()
                results_df["Prediction"] = labels
                results_df["Confidence"] = confidence

                st.success("✅ Batch prediction completed successfully!")
                st.markdown("<h4>🧾 Prediction Results</h4>", unsafe_allow_html=True)
                st.dataframe(results_df)

                csv = results_df.to_csv(index=False).encode('utf-8')
                st.download_button(
                    label="📥 Download Results as CSV",
                    data=csv,
                    file_name="thyroid_batch_predictions.csv",
                    mime="text/csv",
                )
        except Exception as e:
            st.error(f"Error processing file: {e}")
    else:
        st.info("Please upload a CSV file to begin batch prediction.")

# -------------------------------------------------------
# ABOUT PAGE
# -------------------------------------------------------
with tabs[3]:
    st.markdown("<h1 style='text-align:center; color:#228B22; margin-bottom:10px;'>About This Application</h1>", unsafe_allow_html=True)
    st.markdown("<hr style='border:1px solid #ddd'>", unsafe_allow_html=True)

    st.markdown("""
The Thyroid Disease Detection System is an advanced, machine learning-driven web application designed to assist in the early identification and preliminary evaluation of thyroid disorders. By analyzing critical biochemical markers obtained from routine blood tests including TSH, T3, TT4, T4U, and FTI. The system provides clear insights into whether an individual is likely to have hypothyroidism, hyperthyroidism, or normal thyroid function.

Beyond basic detection, the platform emphasizes precision, interpretability, and usability. It transforms raw clinical data into actionable insights, helping users quickly understand thyroid health status without the need for extensive manual calculations or deep medical expertise.

This solution represents the integration of artificial intelligence into clinical decision support systems, demonstrating how predictive modeling can complement traditional laboratory diagnostics. By leveraging data-driven algorithms, the system enables healthcare professionals, diagnostic laboratories, and research institutions to make more informed, evidence-based evaluations. It serves as a supportive tool for screening, monitoring, and patient education, empowering both clinicians and individuals to act proactively in managing thyroid health.

    """, unsafe_allow_html=True)

    st.markdown("<h2>System Overview</h2>", unsafe_allow_html=True )
    st.markdown("""
he platform is designed to streamline the analysis of critical thyroid-related parameters, including TSH, T3, TT4, T4U, and FTI, alongside patient-specific information such as age, sex, medical history, and other relevant clinical factors. By leveraging a trained machine learning model, the system accurately classifies patient data into distinct diagnostic categories, enabling early detection of hypothyroidism, hyperthyroidism, or normal thyroid function.

The application offers two primary functionalities to accommodate different user needs:

1. Single Prediction: Users can manually input individual patient values to obtain an immediate diagnostic prediction, along with an associated confidence score indicating the reliability of the model’s output. This feature is ideal for quick, on-the-spot evaluations or one-off clinical assessments.

2. Batch Prediction: Users can upload a CSV file containing multiple patient records to perform simultaneous predictions in bulk. The system processes each record efficiently, providing both predicted thyroid condition and the corresponding confidence level for each patient. This functionality is particularly useful for laboratories, research teams, or clinical settings managing large datasets.

Overall, the platform combines speed, accuracy, and interpretability, allowing healthcare professionals, researchers, and students to gain actionable insights while supporting informed decision-making and proactive patient care.

    """, unsafe_allow_html=True)

    st.markdown("<h2>Core Technologies</h2>", unsafe_allow_html=True)
    st.markdown("""
To build a robust, interactive, and accurate thyroid detection system, the application leverages a set of proven tools and libraries for web development, data processing, and machine learning:

Python & Streamlit for interactive web application development
Python provides a versatile and widely-used programming environment, ideal for data analysis, machine learning, and backend development. Streamlit, built on Python, allows for the rapid creation of highly interactive web applications. It enables users to input data, run predictions, visualize results, and navigate features effortlessly, without needing extensive front-end development skills. Together, they make the platform accessible, user-friendly, and highly responsive.

Scikit-learn for machine learning modeling and evaluation
Scikit-learn is a powerful and well-established machine learning library in Python. It supports a wide range of supervised and unsupervised algorithms, model evaluation metrics, and preprocessing tools. In this system, Scikit-learn is used to train, validate, and deploy predictive models that classify thyroid disorders based on biochemical and patient-specific parameters. Its robust API ensures reproducibility, interpretability, and efficient computation.

Pandas & NumPy for data analysis and transformation
Pandas provides efficient data structures for handling tabular datasets, allowing easy manipulation, cleaning, and transformation of patient records. NumPy offers fast, vectorized operations on numerical data, making computations on lab values and feature scaling highly efficient. Together, these libraries ensure accurate preprocessing, seamless integration of single and batch inputs, and preparation of data in the format required by machine learning models.

Joblib for efficient model persistence and deployment
Joblib allows models, scalers, and other Python objects to be saved and loaded efficiently. This enables the trained machine learning models to be persisted to disk and quickly reloaded into the application without retraining. By leveraging Joblib, the system ensures faster startup, consistent predictions, and easy deployment across different environments, making it reliable for both single-patient and batch-prediction workflows.

    """, unsafe_allow_html=True)


    st.markdown("<h2>Design Philosophy and Objective</h2>", unsafe_allow_html=True)
    st.markdown("""
The design of this system emphasizes three key principles: accuracy, interpretability, and ease of use. Accurate predictions ensure that the outputs are clinically meaningful and can be trusted by healthcare professionals and users alike. Interpretability allows users to understand not just the results, but also the rationale behind the predictions, fostering confidence in the system’s recommendations. Simplicity of use ensures that the platform is accessible to both medical practitioners and non-expert users, minimizing barriers to adoption.

In healthcare applications, reliable predictions and clear, actionable insights are critical. Misinterpretation of data or ambiguous results can lead to delays in diagnosis or inappropriate interventions. By providing precise, comprehensible outputs, this system supports informed decision-making and helps users take timely action.

This system is designed to complement traditional laboratory diagnostics, rather than replace them. By leveraging predictive analytics, it can quickly analyze biochemical parameters, identify patterns, and highlight potential thyroid dysfunctions. This accelerates early detection, guides further testing, and empowers clinicians to prioritize cases that require immediate attention.

The tool is intended for use in research, education, and clinical support environments. It serves as a learning resource for students and practitioners, a supplementary tool for clinicians, and a platform for exploring AI-assisted diagnostics. By improving early detection and streamlining preliminary assessments, it enhances diagnostic efficiency and reduces the cognitive load on healthcare providers.

It is important to note that this system is not a substitute for professional medical evaluation or laboratory testing. All predictions and recommendations are intended to augment human expertise, providing data-driven insights that support, rather than replace, clinical judgment. Users should always consult licensed healthcare professionals for definitive diagnosis and treatment decisions.
    """, unsafe_allow_html=True)



# -------------------------------------------------------
# LEARN PAGE
# -------------------------------------------------------
with tabs[4]:
    st.markdown("<h1 style='text-align:center; color:#228B22; margin-bottom:10px;'>About REMI - The Thyroid Disease Detector</h1>", unsafe_allow_html=True)
    st.markdown("<hr style='border:1px solid #ddd'>", unsafe_allow_html=True)
    # custom styling for headings, text color, and boxes
    st.markdown("""
        <style>
        h2, h3, h4 {
            color: #015A84 !important;
        }
        body, p, li, td, th {
            color: #1E293B !important;  /* deep gray-blue text */
        }
        .info-box {
            border: 1px solid #000;
            border-radius: 10px;
            background-color: rgba(255, 255, 255, 0.6);
            padding: 15px;
            margin-top: 10px;
            margin-bottom: 20px;
        }
        table {
            width: 100%;
            border-collapse: collapse;
            margin-top: 10px;
            color: #1E293B;
        }
        th, td {
            border: 1px solid #000;
            padding: 8px;
            text-align: left;
        }
        th {
            background-color: rgba(1, 90, 132, 0.1);
            color: #015A84;
        }
        ul {
            color: #1E293B;
        }
        </style>
    """, unsafe_allow_html=True)

    st.markdown("<h2>Learn About Thyroid Disorders</h2>", unsafe_allow_html=True)
    st.markdown("""
    The thyroid gland plays a vital role in regulating metabolism, energy, and hormone balance.
    Understanding thyroid disorders can help identify symptoms early and improve management.

    ### Common Thyroid Conditions
    - Hypothyroidism: The thyroid produces too little hormone.
        Symptoms: fatigue, weight gain, cold intolerance, depression.
    - Hyperthyroidism: The thyroid produces too much hormone.
        Symptoms: weight loss, anxiety, tremors, heat intolerance.
    - Goitre: Enlargement of the thyroid gland, can occur with or without hormone imbalance.
    """, unsafe_allow_html=True)

    # Common Symptoms (no box)
    st.markdown("""
    <h3>Common Symptoms of Thyroid Imbalance</h3>
    <table>
    <tr><th>Type</th><th>Symptoms</th></tr>
    <tr><td>Hypothyroidism</td><td>Fatigue, dry skin, weight gain, hair loss, feeling cold, slow heart rate</td></tr>
    <tr><td>Hyperthyroidism</td><td>Weight loss, irritability, sweating, rapid heartbeat, trembling hands</td></tr>
    </table>
    """, unsafe_allow_html=True)

    # Key Blood Tests box
    st.markdown("""
    <div class="info-box">
    <h3>Key Blood Tests</h3>
    <table>
        <tr>
            <th>Test</th><th>Measures</th><th>Typical Range</th><th>Meaning</th>
        </tr>
        <tr>
            <td><i>TSH</i></td><td>Thyroid Stimulating Hormone</td><td>0.4 – 4.0 mIU/L</td><td>High = Hypothyroid; Low = Hyperthyroid</td>
        </tr>
        <tr>
            <td><i>T3</i></td><td>Triiodothyronine</td><td>0.8 – 2.0 ng/mL</td><td>Active thyroid hormone</td>
        </tr>
        <tr>
            <td><i>TT4</i></td><td>Total Thyroxine</td><td>60 – 150 µg/dL</td><td>Total circulating T4</td>
        </tr>
        <tr>
            <td><i>T4U</i></td><td>Thyroxine Uptake</td><td>0.7 – 1.8</td><td>Measures T4-binding capacity</td>
        </tr>
        <tr>
            <td><i>FTI</i></td><td>Free Thyroxine Index</td><td>80 – 180</td><td>Estimates free thyroid hormone levels</td>
        </tr>
    </table>
    </div>
    """, unsafe_allow_html=True)

    # Thyroid diet box
    st.markdown("""
    <div class="info-box">
    <h3>Healthy Diet for Thyroid Health (Indian Menu)</h3>
    A balanced diet supports thyroid function and helps manage symptoms effectively.
    Focus on nutrient-rich foods and avoid those that interfere with hormone absorption.

    <h4>What to Include</h4>
    <table>
        <tr><th>Category</th><th>Examples</th></tr>
        <tr><td>Iodine-rich foods</td><td>Iodized salt, seaweed (moderation), curd</td></tr>
        <tr><td>Protein sources</td><td>Moong dal, lentils, paneer, eggs, chicken, fish</td></tr>
        <tr><td>Selenium & Zinc</td><td>Sunflower seeds, pumpkin seeds, cashews, whole grains</td></tr>
        <tr><td>Fibre & Antioxidants</td><td>Fruits (papaya, apple, pomegranate), vegetables (palak, lauki, beans)</td></tr>
        <tr><td>Healthy fats</td><td>Ghee (in moderation), olive oil, nuts, seeds</td></tr>
    </table>

    <h4>Foods to Limit or Avoid</h4>
    <table>
        <tr><th>Type</th><th>Examples</th></tr>
        <tr><td>Goitrogenic foods (excess)</td><td>Raw cabbage, cauliflower, broccoli (cook before eating)</td></tr>
        <tr><td>Soy products</td><td>Soy chunks, tofu, soy milk</td></tr>
        <tr><td>Refined sugar & carbs</td><td>Pastries, sweets, white bread</td></tr>
        <tr><td>Caffeine & alcohol</td><td>Limit intake to avoid absorption issues</td></tr>
    </table>

    <h4>Sample One-Day Thyroid-Friendly Meal Plan</h4>
    <table>
        <tr><th>Time</th><th>Meal</th></tr>
        <tr><td>Morning (empty stomach)</td><td>Thyroid medication + lukewarm water</td></tr>
        <tr><td>Breakfast</td><td>Vegetable oats upma + 1 boiled egg / glass of milk</td></tr>
        <tr><td>Mid-morning</td><td>Fruit bowl (papaya or apple)</td></tr>
        <tr><td>Lunch</td><td>Brown rice / phulka + dal + palak sabzi + curd</td></tr>
        <tr><td>Evening</td><td>Roasted chana / handful of nuts</td></tr>
        <tr><td>Dinner</td><td>Moong dal khichdi / grilled paneer + sautéed veggies</td></tr>
        <tr><td>Before bed</td><td>Warm turmeric milk (optional)</td></tr>
    </table>
    </div>
    """, unsafe_allow_html=True)

    # Everyday healthy diet box
    st.markdown("""
    <div class="info-box">
    <h3>Everyday Healthy Diet (Indian Style)</h3>
    <p>Maintaining a balanced diet helps support energy, immunity, and long-term wellness.</p>

    <h4>Basic Guidelines</h4>
    <ul>
        <li>Eat fresh, home-cooked meals with balanced portions.</li>
        <li>Include all food groups: whole grains, proteins, healthy fats, fruits, and vegetables.</li>
        <li>Avoid overeating, processed snacks, and sugary drinks.</li>
        <li>Stay hydrated and active throughout the day.</li>
    </ul>

    <h4>Ideal Daily Indian Meal Plan</h4>
    <table>
        <tr><th>Time</th><th>Meal</th></tr>
        <tr><td>Early Morning</td><td>Warm water with lemon / soaked almonds (4–5)</td></tr>
        <tr><td>Breakfast</td><td>Poha / vegetable paratha with curd / idli with sambar</td></tr>
        <tr><td>Mid-Morning</td><td>Seasonal fruit (banana, apple, orange)</td></tr>
        <tr><td>Lunch</td><td>Brown rice / 2 rotis + dal + sabzi + salad + curd</td></tr>
        <tr><td>Evening Snack</td><td>Sprouts chaat / roasted makhana / green tea</td></tr>
        <tr><td>Dinner</td><td>Light meal – khichdi / vegetable soup + grilled paneer or tofu</td></tr>
        <tr><td>Before Bed</td><td>Warm milk with turmeric (optional)</td></tr>
    </table>

    <h4>Healthy Habits</h4>
    <ul>
        <li>Drink 8–10 glasses of water daily.</li>
        <li>Limit fried food, sugar, and packaged juices.</li>
        <li>Practice mindful eating — eat slowly and avoid screens.</li>
        <li>Get 7–8 hours of quality sleep.</li>
    </ul>
    </div>
    """, unsafe_allow_html=True)

    # Quick Do’s & Don’ts (no box)
    st.markdown("""
    <h3>Quick Do’s & Don’ts</h3>
    <table>
    <tr><th>Do</th><th>Don’t</th></tr>
    <tr><td>Take thyroid medication on an empty stomach</td><td>Skip doses or take with coffee/milk</td></tr>
    <tr><td>Eat iodine and selenium-rich foods</td><td>Overconsume raw cruciferous veggies</td></tr>
    <tr><td>Exercise moderately</td><td>Ignore fatigue or sudden mood swings</td></tr>
    </table>
    """, unsafe_allow_html=True)

    # Disclaimer
    st.markdown("""
    ### Medical Disclaimer
    This app is for educational and research purposes only.
    It should not be used for self-diagnosis or treatment decisions.
    Always consult a licensed healthcare provider for clinical interpretation.
    """, unsafe_allow_html=True)



# -------------------------------------------------------
# WELLNESS HUB PAGE
# -------------------------------------------------------
with tabs[5]:
    st.markdown("<h1 style='text-align:center; color:#228B22; margin-bottom:10px;'>Wellness Hub – Find Trusted Thyroid Specialists in India</h1>", unsafe_allow_html=True)
    st.markdown("<hr style='border:1px solid #ddd'>", unsafe_allow_html=True)
    st.markdown("""
        <p style='font-size:16px; color:#475569;'>
            Discover top hospitals and endocrinology specialists across India specializing in thyroid care and hormonal health.
            Use this directory to find reliable medical consultation and advanced diagnostic facilities.
        </p>
    """, unsafe_allow_html=True)

    hospitals = [
        {
            "city": "New Delhi",
            "hospital": "All India Institute of Medical Sciences (AIIMS)",
            "doctor": "Dr. Ramesh Goyal – Endocrinologist",
            "address": "Ansari Nagar, New Delhi – 110029",
            "contact": "+91 11 2658 8500"
        },
        {
            "city": "Mumbai",
            "hospital": "Fortis Hospital, Mulund",
            "doctor": "Dr. Shashank Joshi – Senior Consultant, Endocrinology",
            "address": "Mulund Goregaon Link Rd, Mumbai – 400078",
            "contact": "+91 22 6799 3333"
        },
        {
            "city": "Chennai",
            "hospital": "Apollo Hospital",
            "doctor": "Dr. V. Mohan – Chief Diabetologist & Endocrinologist",
            "address": "Greams Lane, Off Greams Road, Chennai – 600006",
            "contact": "+91 44 2829 3333"
        },
        {
            "city": "Bengaluru",
            "hospital": "Manipal Hospital",
            "doctor": "Dr. Suresh Raghavan – Consultant Endocrinologist",
            "address": "Old Airport Road, Bengaluru – 560017",
            "contact": "+91 80 2222 1111"
        },
        {
            "city": "Kolkata",
            "hospital": "AMRI Hospitals",
            "doctor": "Dr. Anirban Majumder – Endocrinologist",
            "address": "JC-16 & 17, Salt Lake City, Kolkata – 700098",
            "contact": "+91 33 6680 0000"
        },
        {
            "city": "Hyderabad",
            "hospital": "Yashoda Hospitals",
            "doctor": "Dr. Nitin Kapoor – Consultant Endocrinologist",
            "address": "Raj Bhavan Road, Somajiguda, Hyderabad – 500082",
            "contact": "+91 40 4567 4567"
        },
        {
            "city": "Pune",
            "hospital": "Ruby Hall Clinic",
            "doctor": "Dr. Rajesh Rajput – Thyroid & Hormone Specialist",
            "address": "40, Sassoon Road, Pune – 411001",
            "contact": "+91 20 2616 3391"
        },
        {
            "city": "Ahmedabad",
            "hospital": "Sterling Hospital",
            "doctor": "Dr. Harshal Joshi – Consultant Endocrinologist",
            "address": "Gurukul Road, Memnagar, Ahmedabad – 380052",
            "contact": "+91 79 4001 1111"
        }
    ]

    # Style the cards
    st.markdown("""
        <style>
        .hospital-card {
            background-color: #F1EDEA;
            border-radius: 12px;
            padding: 18px 22px;
            margin-bottom: 16px;
            box-shadow: 0 2px 6px rgba(0,0,0,0.08);
            border: 1px solid #e2e8f0;
        }
        .hospital-card h4 {
            margin: 0;
            color: #1e3a8a;
        }
        .hospital-card p {
            margin: 4px 0;
            color: #334155;
        }
        </style>
    """, unsafe_allow_html=True)

    for entry in hospitals:
        st.markdown(f"""
            <div class='hospital-card'>
                <h4>{entry['hospital']} ({entry['city']})</h4>
                <p><b>Specialist:</b> {entry['doctor']}</p>
                <p><b>Address:</b> {entry['address']}</p>
                <p><b>Contact:</b> {entry['contact']}</p>
            </div>
        """, unsafe_allow_html=True)


# -------------------------------------------------------
# CONTACT PAGE
# -------------------------------------------------------
with tabs[6]:
    st.markdown("<h1 style='text-align:center; color:#228B22; margin-bottom:10px;'>Have Questions? Contact Us!</h1>", unsafe_allow_html=True)
    st.markdown("<hr style='border:1px solid #ddd'>", unsafe_allow_html=True)
    st.markdown("""
        <style>
        /* ===== Input fields (Your Name, Your Email) ===== */
        div[data-baseweb="input"] {
            background-color: #F1EDEA !important;
            border-radius: 10px !important;
            border: 1px solid #d1d5db !important;
            box-shadow: 0px 2px 4px rgba(0,0,0,0.05) !important;
        }
        div[data-baseweb="input"] input {
            background-color: #F1EDEA !important;
            color: #1e293b !important;
            border: none !important;
        }

        /* ===== Text area (Your Message) ===== */
        div[data-baseweb="textarea"] {
            background-color: #ffffff !important;   /* white box look */
            border-radius: 10px !important;
            border: 1px solid #d1d5db !important;
            box-shadow: 0px 2px 6px rgba(0,0,0,0.08) !important;
        }
        div[data-baseweb="textarea"] textarea {
            background-color: #ffffff !important;
            color: #1e293b !important;
            border: none !important;
            border-radius: 10px !important;
            padding: 10px !important;
        }

        /* ===== Button styling ===== */
        .stForm button[kind="primary"] {
            background-color: #F1EDEA !important;
            color: #1e293b !important;
            border: 1px solid #cbd5e1 !important;
            border-radius: 10px !important;
            transition: all 0.2s ease-in-out !important;
        }
        .stForm button[kind="primary"]:hover {
            background-color: #e7deda !important;
            color: #111827 !important;
            border-color: #94a3b8 !important;
        }

        /* ===== Header and list styling ===== */
        h2, h3 { color: #1e293b !important; }

        .team-list {
            list-style-type: none;
            padding-left: 0;
        }
        .team-list li {
            background-color: #F1EDEA;
            padding: 12px 18px;
            margin-bottom: 10px;
            border-radius: 10px;
            font-size: 16px;
            color: #1e293b;
            display: flex;
            justify-content: space-between;
            align-items: center;
            border: 1px solid #d1d5db;
            box-shadow: 0 2px 4px rgba(0,0,0,0.05);
        }

        /* ===== LinkedIn button styling ===== */
        .linkedin-btn {
            background-color: #0077B5;
            color: white !important;
            text-decoration: none;
            padding: 6px 12px;
            border-radius: 6px;
            font-size: 14px;
            font-weight: 500;
            transition: all 0.2s ease-in-out;
        }
        .linkedin-btn:hover {
            background-color: #005b86;
        }
        </style>
    """, unsafe_allow_html=True)

    # --- Contact form section ---
    st.markdown("<h2>Contact Us</h2>", unsafe_allow_html=True)
    with st.form("contact_form"):
        name = st.text_input("Your Name")
        email = st.text_input("Your Email")
        message = st.text_area("Your Message")
        submitted = st.form_submit_button("Send Message")
        if submitted:
            st.success("✅ Thank you! Your message has been received.")

    # --- Team section with LinkedIn buttons ---
    st.markdown("""
        <h3>Connect With Us</h3>
        <ul class="team-list">
            <li><span><b>Ishika Das</b> – 2205289</span>
                <a class="linkedin-btn" href="https://www.linkedin.com/in/ishika-das-39aa2324a/" target="_blank">LinkedIn</a>
            </li>
            <li><span><b>Anishka</b> – 2205356</span>
                <a class="linkedin-btn" href="https://www.linkedin.com/in/anishka2011/" target="_blank">LinkedIn</a>
            </li>
            <li><span><b>Sangbarta Sarkar</b> – 22051612</span>
                <a class="linkedin-btn" href="https://www.linkedin.com/in/sangbartasarkar02" target="_blank">LinkedIn</a>
            </li>
            <li><span><b>Aryan Pattnaik</b> – 2228018</span>
                <a class="linkedin-btn" href="https://www.linkedin.com/in/aryan-pattnaik/?originalSubdomain=in" target="_blank">LinkedIn</a>
            </li>
        </ul>
    """, unsafe_allow_html=True)


# --- Footer ---
st.markdown("""
    <hr style='margin-top:50px;'>
    <div style='text-align: center; font-size: 13px; color: #64748b;'>
        © 2025 <b>REMI</b> | Powered by Streamlit and Machine Learning
    </div>
""", unsafe_allow_html=True)

Overwriting thyroid_app.py


In [ ]:
import os
import subprocess
import time
from pyngrok import ngrok

# ---- Kill any old process ----
!kill -9 $(lsof -t -i:8501) 2>/dev/null || echo "No existing process"
ngrok.kill()

# ---- Set Ngrok token ----
NGROK_TOKEN = " Enter your token here "
ngrok.set_auth_token(NGROK_TOKEN)

# ---- Start Streamlit app ----
print("🚀 Starting Streamlit server on port 8501...")
process = subprocess.Popen(["streamlit", "run", "thyroid_app.py", "--server.port", "8501"],
                           stdout=subprocess.PIPE,
                           stderr=subprocess.PIPE)

# ---- Wait for Streamlit to start ----
time.sleep(15)  # wait 15 seconds for Streamlit to boot up

# ---- Create ngrok tunnel ----
public_url = ngrok.connect(8501)
print("✅ Streamlit App is Live at:", public_url.public_url)
print("If the link fails, re-run this cell only (it restarts ngrok and streamlit).")


No existing process
🚀 Starting Streamlit server on port 8501...
✅ Streamlit App is Live at: https://exigeant-elidia-agrostologic.ngrok-free.dev
If the link fails, re-run this cell only (it restarts ngrok and streamlit).
